In [68]:
import pandas as pd
import yfinance as yf
import numpy as np
import datetime
import plotly.io as pio
import plotly.graph_objects as go
import requests
import random
from prettytable import PrettyTable
from collections import deque
from yahoo_fin.stock_info import get_holders
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
from newspaper import Article
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [69]:
# User's choice of stock
choice = 'MSFT'
stock = yf.Ticker(choice)

In [70]:
# Profile

# Company Information
print("Company Information:")
print(f"Name: {stock.info.get('longName')}")
address = ", ".join(filter(None, [stock.info.get('address1'), stock.info.get('city'), stock.info.get('state'), stock.info.get('zip'), stock.info.get('country')]))
print(f"Address: {address}")
print(f"Phone: {stock.info.get('phone')}")
print(f"Website: {stock.info.get('website')}")
print(f"Exchange: {stock.info.get('exchange')}")
print(f"Currency: {stock.info.get('currency')}")
print(f"Full-time Employees: {stock.info.get('fullTimeEmployees')}")
print(f"TimeZone: {stock.info.get('timeZoneFullName')}")
print("Industry:",  stock.info['industry'])
print("Sector:", stock.info['sector'])
print(f"\nBusiness Description:\n{stock.info['longBusinessSummary']}")

Company Information:
Name: Microsoft Corporation
Address: One Microsoft Way, Redmond, WA, 98052-6399, United States
Phone: 425 882 8080
Website: https://www.microsoft.com
Exchange: NMS
Currency: USD
Full-time Employees: 221000
TimeZone: America/New_York
Industry: Software - Infrastructure
Sector: Technology

Business Description:
Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power apps, and power automate; and on-premises ERP and CRM applications. The Intelligent Cloud segm

In [71]:
# Print Company Officers
print("\nKey Executives:")
company_officers = stock.info.get('companyOfficers', [])

# Create a pretty table instance
table = PrettyTable(['Name', 'Title'])

# Add data to the table
for officer in company_officers:
    table.add_row([officer.get('name'), officer.get('title')])

# Print the table
print(table)


Key Executives:
+-----------------------------+-----------------------------------------------------------------------+
|             Name            |                                 Title                                 |
+-----------------------------+-----------------------------------------------------------------------+
|      Mr. Satya  Nadella     |                             Chairman & CEO                            |
|  Mr. Bradford L. Smith LCA  |                       President & Vice Chairman                       |
|       Ms. Amy E. Hood       |                           Executive VP & CFO                          |
|    Mr. Judson B. Althoff    |                Executive VP & Chief Commercial Officer                |
| Mr. Christopher David Young | Executive Vice President of Business Development, Strategy & Ventures |
|      Ms. Alice L. Jolla     |                Corporate VP & Chief Accounting Officer                |
|    Mr. James Kevin Scott    |                

In [72]:
# Charts

# Stock Price Chart
labels = ["1 day", "5 days", "1 month", "6 months", "year to date", "1 year", "5 years", "max"]
period_values = ["1d", "5d", "1mo", "6mo", "ytd", "1y", "5y", "max"]

# Randomly select one period
selected_period_index = random.randint(0, len(period_values) - 1)
selected_period_value = period_values[selected_period_index]

if selected_period_value == '1d':
    data = yf.download(tickers=choice, period=selected_period_value, interval='60m')
else:
    data = yf.download(tickers=choice, period=selected_period_value)
    
# Create figure
fig = go.Figure()

# Add trace for closing price
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Close', 
                         hoverinfo='x+y+text', 
                         text=['Open: {:.2f}<br>High: {:.2f}<br>Low: {:.2f}<br>Close: {:.2f}<br>Volume: {:.0f}'.format(open_val, high_val, low_val, close_val, volume_val) 
                               for open_val, high_val, low_val, close_val, volume_val in zip(data['Open'], data['High'], data['Low'], data['Close'], data['Volume'])]))

# Update layout
fig.update_layout(
    title=f"{choice} Stock Data for {labels[selected_period_index]}",
    xaxis_title="Time",
    yaxis_title="Price",
    height=400,
    width=800,
)

# Show plot
fig.show()

[*********************100%%**********************]  1 of 1 completed


In [73]:
# Simple Moving Averages Chart
def fetch_data(symbol, interval, outputsize='full'):
    """
    Fetch historical price data from Alpha Vantage API
    """
    api_key = '4Z6YC4MI9GIWXELC'
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_{interval.upper()}&symbol={symbol}&apikey={api_key}&outputsize={outputsize}'
    response = requests.get(url)
    data = response.json()
    return data

def calculate_sma(data, window, interval, time_series_key):
    """
    Calculate Simple Moving Average (SMA) for the given window and interval
    """
    try:
        if time_series_key not in data:
            raise KeyError(f"Key '{time_series_key}' not found in data dictionary.")
        
        df = pd.DataFrame(data[time_series_key]).T
        df.index = pd.to_datetime(df.index)
        df['Close'] = df['4. close'].astype(float)
        
        sma = df['Close'].rolling(window=window).mean()
        return sma
    
    except (KeyError, ValueError) as e:
        print(f"Error: {e}")
        return None
    
def plot_prices_with_sma(symbol, data, sma_50, sma_200, interval, time_series_key):
    """
    Plot historical prices with SMA using Plotly
    """
    df = pd.DataFrame(data[time_series_key]).T
    df.index = pd.to_datetime(df.index)
    df['Close'] = df['4. close'].astype(float)

    trace_price = go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Price')
    trace_sma_50 = go.Scatter(x=df.index, y=sma_50, mode='lines', name='50-day SMA')
    trace_sma_200 = go.Scatter(x=df.index, y=sma_200, mode='lines', name='200-day SMA')

    layout = go.Layout(title=f'{symbol} Historical Prices with SMA ({interval.capitalize()})', xaxis=dict(title='Date'), yaxis=dict(title='Price'))
    fig = go.Figure(data=[trace_price, trace_sma_50, trace_sma_200], layout=layout)
    fig.show()

window_50 = 50
window_200 = 200

interval = random.choice(['daily', 'weekly', 'monthly'])
if interval.lower() == 'daily':
    time_series_key = 'Time Series (Daily)'
elif interval.lower() == 'weekly' or interval.lower() == 'monthly':
    time_series_key = f'{interval.capitalize()} Time Series'
    
data = fetch_data(choice, interval)
sma_50 = calculate_sma(data, window_50, interval, time_series_key)
sma_200 = calculate_sma(data, window_200, interval, time_series_key)

plot_prices_with_sma(choice, data, sma_50, sma_200, interval, time_series_key)

In [74]:
# Bollinger Bands Chart
def get_bollinger_bands(symbol, interval, time_period=20, series_type='close'):
    api_key = '4Z6YC4MI9GIWXELC'  # Replace 'YOUR_API_KEY' with your Alpha Vantage API key
    endpoint = f'https://www.alphavantage.co/query?function=BBANDS&symbol={symbol}&interval={interval}&time_period={time_period}&series_type={series_type}&apikey={api_key}'

    try:
        response = requests.get(endpoint)
        data = response.json()
        
        # Extract Bollinger Bands data
        dates = list(data['Technical Analysis: BBANDS'].keys())
        upper_band = [float(data['Technical Analysis: BBANDS'][date]['Real Upper Band']) for date in dates]
        middle_band = [float(data['Technical Analysis: BBANDS'][date]['Real Middle Band']) for date in dates]
        lower_band = [float(data['Technical Analysis: BBANDS'][date]['Real Lower Band']) for date in dates]

        # Create traces for Bollinger Bands
        trace_upper = go.Scatter(x=dates, y=upper_band, mode='lines', name='Upper Band')
        trace_middle = go.Scatter(x=dates, y=middle_band, mode='lines', name='Middle Band')
        trace_lower = go.Scatter(x=dates, y=lower_band, mode='lines', name='Lower Band')

        # Create plot layout
        layout = go.Layout(title=f'Bollinger Bands for {symbol}', xaxis=dict(title='Date'), yaxis=dict(title='Price'))

        # Create figure and plot
        fig = go.Figure(data=[trace_upper, trace_middle, trace_lower], layout=layout)
        fig.show()

    except Exception as e:
        print(f"Error occurred: {e}")
        
interval = random.choice(['daily', 'weekly', 'monthly'])
get_bollinger_bands(choice, interval)

In [75]:
# Relative Strength Index (RSI) Chart
def get_rsi_data(symbol, api_key, interval, time_period):
    url = f"https://api.polygon.io/v1/indicators/rsi/{symbol}?timespan={interval}&window={time_period}&series_type=close&order=desc&apiKey={api_key}"
    response = requests.get(url)
    data = response.json()
    
    if 'results' in data and 'values' in data['results']:
        rsi_values = data['results']['values']
        timestamps = [value['timestamp'] for value in rsi_values]
        rsi = [value['value'] for value in rsi_values]
        
        df = pd.DataFrame({'timestamp': timestamps, 'rsi': rsi})
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)  # Set timestamp as index
        return df
    else:
        print("No RSI data found in the response.")
        return None

def compute_rsi(data, time_period):
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(time_period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(time_period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def plot_rsi(df, symbol):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df['rsi'], mode='lines', name='RSI'))
    fig.update_layout(
        title=f'Relative Strength Index (RSI) for {symbol}',
        xaxis_title='Date',
        yaxis_title='RSI',
        height=600,
        width=1000,
    )
    fig.show()


api_key = 'V2jiIqWR6ZKdcV0BC_sh84w6ZuaxArdE'  # Your Polygon API key
time_period = 14  # Number of data points used to calculate RSI
valid_intervals = ['day', 'week', 'month']
interval = random.choice(valid_intervals)
# Get RSI data
rsi_df = get_rsi_data(choice, api_key, interval, time_period=time_period)

# Plot RSI
plot_rsi(rsi_df, choice)

In [76]:
def format_large_number(number):
    try:
        parts = str(number).split('.')
        parts[0] = "{:,}".format(int(parts[0]))
        return '.'.join(parts)
    except (ValueError, TypeError):
        return str(number)

In [77]:
# Insights

# Basic Information
print("\nBasic Information:")
print(f"Market Cap: {format_large_number(stock.info.get('marketCap'))}")
print(f"Previous Close: {stock.info.get('previousClose')}")
print(f"Current Price: {stock.info.get('currentPrice')}")
print(f"52-Week Range: {stock.info.get('fiftyTwoWeekLow')}-{stock.info.get('fiftyTwoWeekHigh')}")
print(f"Average Volume: {format_large_number(stock.info.get('averageVolume'))}")

# Dividends and Splits
print("\nDividends and Splits:")
print(f"Dividend Rate: {stock.info.get('dividendRate')}")
print(f"Payout Ratio: {stock.info.get('payoutRatio')}")
print(f"Dividend Yield: {stock.info.get('dividendYield')}")
print(f"Last Dividend Value: {stock.info.get('lastDividendValue')}") 
print(f"Ex-Dividend Date: {datetime.datetime.fromtimestamp(stock.info.get('exDividendDate')).strftime('%d/%m/%Y')}")
print(f"Last Split Factor: {stock.info.get('lastSplitFactor')}")  
print(f"Last Split Date: {datetime.datetime.fromtimestamp(stock.info.get('lastSplitDate')).strftime('%d/%m/%Y')}")  
print(f"Last Dividend Date: {datetime.datetime.fromtimestamp(stock.info.get('lastDividendDate')).strftime('%d/%m/%Y')}")  
print(f"Five Year Average Dividend Yield: {stock.info.get('fiveYearAvgDividendYield')}")

# Earnings and Revenue
print("\nEarnings and Revenue:")
print(f"Trailing EPS (Earnings Per Share): {stock.info.get('trailingEps')}")
print(f"Forward EPS (Earnings Per Share): {stock.info.get('forwardEps')}")
print(f"Total Revenue: {format_large_number(stock.info.get('totalRevenue'))}")
print(f"Earnings Growth: {stock.info.get('earningsGrowth')}")
print(f"Earnings Quarterly Growth: {stock.info.get('earningsQuarterlyGrowth')}")
print(f"Revenue Growth:  {stock.info.get('revenueGrowth')}")

# Ratios and Margins
print("\nRatios and Margins:")
print(f"Trailing P/E Ratio: {stock.info.get('trailingPE')}")
print(f"Forward P/E Ratio: {stock.info.get('forwardPE')}")
print(f"Profit Margins: {stock.info.get('profitMargins')}")
print(f"Return on Equity: {stock.info.get('returnOnEquity')}")
print(f"Return on Assets: {stock.info.get('returnOnAssets')}")
print(f"Debt-to-Equity Ratio: {stock.info.get('debtToEquity')}")
print(f"Gross Margins: {stock.info.get('grossMargins')}")
print(f"EBITDA Margins: {stock.info.get('ebitdaMargins')}")
print(f"Operating Margins: {stock.info.get('operatingMargins')}")
print(f"Trailing PEG Ratio: {stock.info.get('trailingPegRatio')}")
print(f"PEG Ratio: {stock.info.get('pegRatio')}") 

# Cash Flow and Enterprise Value
print("\nCash Flow and Enterprise Value:")
print(f"Total Cash: {format_large_number(stock.info.get('totalCash'))}")
print(f"Free Cash Flow: {format_large_number(stock.info.get('freeCashflow'))}")
print(f"EBITDA (Earnings Before Interest, Taxes, Depreciation, and Amortization): {format_large_number(stock.info.get('ebitda'))}")
print(f"Operating Cash Flow: {format_large_number(stock.info.get('operatingCashflow'))}")
print(f"Enterprise Value: {format_large_number(stock.info.get('enterpriseValue'))}")
print(f"Enterprise to Revenue Ratio: {stock.info.get('enterpriseToRevenue')}")
print(f"Enterprise to EBITDA Ratio: {stock.info.get('enterpriseToEbitda')}")

# Other Metrics
print("\nOther Metrics:")
print(f"Book Value: {stock.info.get('bookValue')}")
print(f"Price-to-Book (P/B) Ratio: {stock.info.get('priceToBook')}")
print(f"Revenue Per Share: {stock.info.get('revenuePerShare')}")
print(f"Total Cash Per Share: {stock.info.get('totalCashPerShare')}")
print(f"Average Daily Volume 10-Day: {format_large_number(stock.info.get('averageDailyVolume10Day'))}")
print(f"Regular Market Volume: {format_large_number(stock.info.get('regularMarketVolume'))}")
print(f"Regular Market Day Low: {stock.info.get('regularMarketDayLow')}")
print(f"Regular Market Day High: {stock.info.get('regularMarketDayHigh')}")
print(f"Regular Market Open: {stock.info.get('regularMarketOpen')}")
print(f"Regular Market Previous Close: {stock.info.get('regularMarketPreviousClose')}")


Basic Information:
Market Cap: 3,117,742,292,992
Previous Close: 420.99
Current Price: 419.485
52-Week Range: 309.45-430.82
Average Volume: 19,926,627

Dividends and Splits:
Dividend Rate: 3.0
Payout Ratio: 0.24780001
Dividend Yield: 0.0070999996
Last Dividend Value: 0.75
Ex-Dividend Date: 15/05/2024
Last Split Factor: 2:1
Last Split Date: 18/02/2003
Last Dividend Date: 15/05/2024
Five Year Average Dividend Yield: 0.94

Earnings and Revenue:
Trailing EPS (Earnings Per Share): 11.54
Forward EPS (Earnings Per Share): 13.26
Total Revenue: 236,583,993,344
Earnings Growth: 0.2
Earnings Quarterly Growth: 0.199
Revenue Growth:  0.17

Ratios and Margins:
Trailing P/E Ratio: 36.350517
Forward P/E Ratio: 31.635368
Profit Margins: 0.36426997
Return on Equity: 0.38487998
Return on Assets: 0.15295
Debt-to-Equity Ratio: 41.963
Gross Margins: 0.69894
EBITDA Margins: 0.52911997
Operating Margins: 0.44588003
Trailing PEG Ratio: 2.0835
PEG Ratio: 2.2

Cash Flow and Enterprise Value:
Total Cash: 80,013,

In [78]:
# Analysts

# Print Analyst Recommendations
print("\nAnalyst Recommendations:")
print(f"Number of Analyst Opinions: {stock.info.get('numberOfAnalystOpinions')}")
print(f"Recommendation Mean: {stock.info.get('recommendationMean')}")
print(f"Recommendation Key: {stock.info.get('recommendationKey')}")
print(f"Target Mean Price: {stock.info.get('targetMeanPrice')}")
print(f"Target High Price: {stock.info.get('targetHighPrice')}")
print(f"Target Low Price: {stock.info.get('targetLowPrice')}")
print(f"Target Median Price: {stock.info.get('targetMedianPrice')}")


Analyst Recommendations:
Number of Analyst Opinions: 48
Recommendation Mean: 1.7
Recommendation Key: buy
Target Mean Price: 472.91
Target High Price: 550.0
Target Low Price: 298.1
Target Median Price: 475.0


In [79]:
# Stock Recommendations Chart
recommendations = stock.recommendations

# Get the current date
current_date = datetime.datetime.today()

# Extract the month from the current date
current_month = current_date.strftime('%B')

# Define the list of months
months = ['December', 'November', 'October', 'September', 'August', 'July', 'June', 'May', 'April', 'March', 'February', 'January']

start_index = months.index(current_month)

# Rotate the list of months in reverse order by -1 index
rotated_months = deque(months)
rotated_months.rotate(-start_index)

# Convert the rotated deque back to a list
custom_labels = list(rotated_months)

# Create traces for each recommendation category
trace_strong_buy = go.Bar(x=custom_labels, y=recommendations['strongBuy'], name='Strong Buy')
trace_buy = go.Bar(x=custom_labels, y=recommendations['buy'], name='Buy')
trace_hold = go.Bar(x=custom_labels, y=recommendations['hold'], name='Hold')
trace_sell = go.Bar(x=custom_labels, y=recommendations['sell'], name='Sell')
trace_strong_sell = go.Bar(x=custom_labels, y=recommendations['strongSell'], name='Strong Sell')

# Create layout
layout = go.Layout(title='Stock Recommendations Chart',
                   xaxis=dict(title='Month'),
                   yaxis=dict(title='Number of Recommendations'))

# Create figure
fig = go.Figure(data=[trace_strong_buy, trace_buy, trace_hold, trace_sell, trace_strong_sell], layout=layout)

# Show plot
fig.show()

In [80]:
# Earnings Calendar
dates = list(stock.calendar.keys())
earnings = list(stock.calendar.values())

# Create a PrettyTable object
table = PrettyTable()
table.field_names = ["Date", "Earnings"]

# Add data to the table
earning_dates = []
for date, earning in zip(dates, earnings):
    if isinstance(earning, list):
        for element in earning:
            if isinstance(element, datetime.date):
                earning_dates.append(element.strftime("%d/%m/%Y"))
            else:
                earning_dates.append(str(element)) 
        table.add_row([date, ", ".join(earning_dates)])
    elif isinstance(earning, datetime.date):
        table.add_row([date, earning.strftime("%d/%m/%Y")])
    elif isinstance(earning, int):
        table.add_row([date, format_large_number(earning)])
    else:
        table.add_row([date, str(earning)])

# Print the table
print(table)

+------------------+------------------------+
|       Date       |        Earnings        |
+------------------+------------------------+
|  Dividend Date   |       13/06/2024       |
| Ex-Dividend Date |       15/05/2024       |
|  Earnings Date   | 23/07/2024, 29/07/2024 |
|  Earnings High   |          3.31          |
|   Earnings Low   |          2.86          |
| Earnings Average |          2.93          |
|   Revenue High   |     65,543,000,000     |
|   Revenue Low    |     64,000,000,000     |
| Revenue Average  |     64,351,800,000     |
+------------------+------------------------+


In [81]:
# Risks

# Print Market Volatility and Performance
print("\nMarket Volatility and Performance:")
print(f"Beta: {stock.info.get('beta')}")
print(f"52-Week Change: {stock.info.get('52WeekChange')}")
print(f"S&P 52-Week Change: {stock.info.get('SandP52WeekChange')}")

# Print Short Interest and Ownership
print("\nShort Interest and Ownership:")
print(f"Short Percent of Float: {stock.info.get('shortPercentOfFloat')}")
print(f"Shares Short: {stock.info.get('sharesShort')}")
print(f"Shares Percent Shares Out: {stock.info.get('sharesPercentSharesOut')}")
print(f"Shares Short Prior Month: {stock.info.get('sharesShortPriorMonth')}")
print(f"Shares Short Prior Month Date: {stock.info.get('sharesShortPriorMonthDate')}")
print(f"Shares Short Previous Month Date: {format_large_number(stock.info.get('sharesShortPreviousMonthDate'))}")
print(f"Held Percent Insiders: {stock.info.get('heldPercentInsiders')}")
print(f"Held Percent Institutions: {stock.info.get('heldPercentInstitutions')}")
print(f"Institutional Ownership: {stock.info.get('institutionalOwnership')}")

# Print Governance and Audit Risk
print("\nGovernance and Audit Risk:")
print(f"Governance Epoch Date: {datetime.datetime.fromtimestamp(stock.info.get('governanceEpochDate'))}")
print(f"Compensation As Of Epoch Date: {datetime.datetime.fromtimestamp(stock.info.get('compensationAsOfEpochDate'))}")
print(f"Audit Risk: {stock.info.get('auditRisk')}")
print(f"Board Risk: {stock.info.get('boardRisk')}")
print(f"Compensation Risk: {stock.info.get('compensationRisk')}")
print(f"Shareholder Rights Risk: {stock.info.get('shareHolderRightsRisk')}")
print(f"Overall Risk: {stock.info.get('overallRisk')}")


Market Volatility and Performance:
Beta: 0.893
52-Week Change: 0.32245398
S&P 52-Week Change: 0.26362717

Short Interest and Ownership:
Short Percent of Float: 0.0067000003
Shares Short: 49773013
Shares Percent Shares Out: 0.0067000003
Shares Short Prior Month: 55771473
Shares Short Prior Month Date: None
Shares Short Previous Month Date: 1,711,584,000
Held Percent Insiders: 0.00054000004
Held Percent Institutions: 0.73648
Institutional Ownership: None

Governance and Audit Risk:
Governance Epoch Date: 2024-05-01 05:30:00
Compensation As Of Epoch Date: 2023-12-31 05:30:00
Audit Risk: 3
Board Risk: 5
Compensation Risk: 2
Shareholder Rights Risk: 2
Overall Risk: 1


In [82]:
# Financials

def extract_date(date_string):
    date_only = date_string.date() if isinstance(date_string, pd.Timestamp) else date_string
    return date_only.strftime("%d/%m/%Y")

In [83]:
# Balance Sheet
balance_sheet = stock.balance_sheet

# Get list of dates from the balance sheet
date_string = list(balance_sheet.keys())

# Define field names for tables including dates
field_names = ['Component'] + [extract_date(date) for date in date_string]

# Initialize dictionaries to store data for each year
main_components_yearly = {extract_date(date): {} for date in date_string}

# Define keys for additional components
main_components = ['Total Capitalization', 'Common Stock Equity', 'Capital Lease Obligations', 'Net Tangible Assets', 'Working Capital',
                   'Invested Capital', 'Tangible Book Value', 'Total Debt', 'Net Debt', 'Share Issued', 'Ordinary Shares Number', 
                   'Treasury Shares Number']

# Iterate over balance sheet data to extract additional components
for date in date_string:
    for component in main_components:
        # Try to extract component value for the current date
        try:
            main_components_yearly[extract_date(date)][component] = format_large_number(balance_sheet[pd.Timestamp(date)][component])
        # Handle the case when the component key is not found for the current date
        except KeyError:
            main_components_yearly[extract_date(date)][component] = None

# PrettyTable for additional components
main_components_table = PrettyTable()
main_components_table.field_names = field_names

# Add data to the table for each component
for component in main_components:
    data_row = [component] + [main_components_yearly[date][component] for date in main_components_yearly]
    main_components_table.add_row(data_row)

# Print the table for additional components
print("\nMain Balance Sheet Components:")
print(main_components_table)


Main Balance Sheet Components:
+---------------------------+-------------------+-------------------+-------------------+-------------------+
|         Component         |     30/06/2023    |     30/06/2022    |     30/06/2021    |     30/06/2020    |
+---------------------------+-------------------+-------------------+-------------------+-------------------+
|    Total Capitalization   | 248,213,000,000.0 | 213,574,000,000.0 | 192,062,000,000.0 | 177,882,000,000.0 |
|    Common Stock Equity    | 206,223,000,000.0 | 166,542,000,000.0 | 141,988,000,000.0 | 118,304,000,000.0 |
| Capital Lease Obligations |  12,728,000,000.0 |  11,489,000,000.0 |  9,629,000,000.0  |  7,671,000,000.0  |
|    Net Tangible Assets    | 128,971,000,000.0 |  87,720,000,000.0 |  84,477,000,000.0 |  67,915,000,000.0 |
|      Working Capital      |  80,108,000,000.0 |  74,602,000,000.0 |  95,749,000,000.0 | 109,605,000,000.0 |
|      Invested Capital     | 253,460,000,000.0 | 216,323,000,000.0 | 200,134,000,000.0 

In [84]:
# Get list of dates
date_string = list(balance_sheet.keys())

# Define field names for tables
field_names = ['Component'] + [extract_date(date) for date in date_string]

# Initialize dictionaries to store data for each year for current and long-term assets
current_assets_yearly = {extract_date(date): {} for date in date_string}
long_term_assets_yearly = {extract_date(date): {} for date in date_string}

# List of components for both current and long-term assets
current_assets_components = ['Cash And Cash Equivalents', 'Other Short Term Investments', 'Accounts Receivable', 
                             'Inventory', 'Hedging Assets Current', 'Other Current Assets']
long_term_assets_components = ['Net PPE', 'Goodwill', 'Other Intangible Assets', 'Investments And Advances', 'Other Non Current Assets']

# Iterate over balance sheet data to extract current and long-term assets
for date in date_string:
    # Extract current assets
    for component in current_assets_components:
        try:
            current_assets_yearly[extract_date(date)][component] = format_large_number(balance_sheet[pd.Timestamp(date)][component])
        except KeyError:
            current_assets_yearly[extract_date(date)][component] = None
    
    # Extract long-term assets
    for component in long_term_assets_components:
        try:
            long_term_assets_yearly[extract_date(date)][component] = format_large_number(balance_sheet[pd.Timestamp(date)][component])
        except KeyError:
            long_term_assets_yearly[extract_date(date)][component] = None

# PrettyTable for current assets
current_assets_table = PrettyTable()
current_assets_table.field_names = field_names

# Add data to the table for each component
for component in current_assets_components:
    data_row = [component] + [current_assets_yearly[date][component] for date in current_assets_yearly]
    current_assets_table.add_row(data_row)

# PrettyTable for long-term assets
long_term_assets_table = PrettyTable()
long_term_assets_table.field_names = field_names

# Add data to the table for each component
for component in long_term_assets_components:
    data_row = [component] + [long_term_assets_yearly[date][component] for date in long_term_assets_yearly]
    long_term_assets_table.add_row(data_row)

# Print the tables
print("Current Assets:")
print(current_assets_table)

print("\nLong-term Assets:")
print(long_term_assets_table)

Current Assets:
+------------------------------+------------------+------------------+-------------------+-------------------+
|          Component           |    30/06/2023    |    30/06/2022    |     30/06/2021    |     30/06/2020    |
+------------------------------+------------------+------------------+-------------------+-------------------+
|  Cash And Cash Equivalents   | 34,704,000,000.0 | 13,931,000,000.0 |  14,224,000,000.0 |  13,576,000,000.0 |
| Other Short Term Investments | 76,552,000,000.0 | 90,818,000,000.0 | 116,032,000,000.0 | 122,951,000,000.0 |
|     Accounts Receivable      | 48,688,000,000.0 | 44,261,000,000.0 |  38,043,000,000.0 |  32,011,000,000.0 |
|          Inventory           | 2,500,000,000.0  | 3,742,000,000.0  |  2,636,000,000.0  |  1,895,000,000.0  |
|    Hedging Assets Current    |   6,000,000.0    |   8,000,000.0    |    78,000,000.0   |        0.0        |
|     Other Current Assets     | 21,807,000,000.0 | 16,924,000,000.0 |  13,393,000,000.0 |  11,4

In [85]:
# Get list of dates from the balance sheet
date_string = list(balance_sheet.keys())

# Define field names for tables including dates
field_names = ['Component'] + [extract_date(date) for date in date_string]

# Initialize dictionaries to store data for each year
current_liabilities_yearly = {extract_date(date): {} for date in date_string}
long_term_liabilities_yearly = {extract_date(date): {} for date in date_string}

# Define components for current liabilities and long-term liabilities
current_liabilities_components = ['Current Debt', 'Current Deferred Revenue', 'Pensionand Other Post Retirement Benefit Plans Current',
                                  'Other Current Liabilities', 'Accounts Payable', 'Income Tax Payable', 'Interest Payable']
long_term_liabilities_components = ['Long Term Debt', 'Employee Benefits', 'Non Current Deferred Revenue', 
                                    'Non Current Deferred Taxes Liabilities', 'Tradeand Other Payables Non Current', 
                                    'Other Non Current Liabilities']

# Iterate over balance sheet data to extract current and long-term liabilities
for date in date_string:
    # Extract current liabilities for each date
    for component in current_liabilities_components:
        try:
            current_liabilities_yearly[extract_date(date)][component] = format_large_number(balance_sheet[pd.Timestamp(date)][component])
        except KeyError:
            current_liabilities_yearly[extract_date(date)][component] = None
    
    # Extract long-term liabilities for each date
    for component in long_term_liabilities_components:
        try:
            long_term_liabilities_yearly[extract_date(date)][component] = format_large_number(balance_sheet[pd.Timestamp(date)][component])
        except KeyError:
            long_term_liabilities_yearly[extract_date(date)][component] = None

# PrettyTable for current liabilities
current_liabilities_table = PrettyTable()
current_liabilities_table.field_names = field_names

# Add data to the table for each component
for component in current_liabilities_components:
    data_row = [component] + [current_liabilities_yearly[date][component] for date in current_liabilities_yearly]
    current_liabilities_table.add_row(data_row)

# PrettyTable for long-term liabilities
long_term_liabilities_table = PrettyTable()
long_term_liabilities_table.field_names = field_names

# Add data to the table for each component
for component in long_term_liabilities_components:
    data_row = [component] + [long_term_liabilities_yearly[date][component] for date in long_term_liabilities_yearly]
    long_term_liabilities_table.add_row(data_row)

# Print the tables with modified titles
print("Current Liabilities:")
print(current_liabilities_table)

print("\nLong-term Liabilities:")
print(long_term_liabilities_table)

Current Liabilities:
+--------------------------------------------------------+------------------+------------------+------------------+------------------+
|                       Component                        |    30/06/2023    |    30/06/2022    |    30/06/2021    |    30/06/2020    |
+--------------------------------------------------------+------------------+------------------+------------------+------------------+
|                      Current Debt                      | 5,247,000,000.0  | 2,749,000,000.0  | 8,072,000,000.0  | 3,749,000,000.0  |
|                Current Deferred Revenue                | 50,901,000,000.0 | 45,538,000,000.0 | 41,525,000,000.0 | 36,000,000,000.0 |
| Pensionand Other Post Retirement Benefit Plans Current | 11,009,000,000.0 | 10,661,000,000.0 | 10,057,000,000.0 | 7,874,000,000.0  |
|               Other Current Liabilities                | 14,745,000,000.0 | 13,067,000,000.0 | 11,666,000,000.0 | 10,027,000,000.0 |
|                    Accounts Paya

In [86]:
# Get list of dates from the balance sheet
date_string = list(balance_sheet.keys())

# Initialize dictionary to store data for each year's stockholders' equity components
stockholders_equity_yearly = {extract_date(date): {} for date in date_string}

# Define stockholders' equity components
stockholders_equity_components = ['Retained Earnings', 'Common Stock', 'Treasury Stock', 'Other Equity Adjustments']

# Iterate over balance sheet data to extract stockholders' equity components
for date in stockholders_equity_yearly:
    for component in stockholders_equity_components:
        try:
            stockholders_equity_yearly[date][component] = format_large_number(balance_sheet[pd.Timestamp(date)][component])
        except KeyError:
            stockholders_equity_yearly[date][component] = None

# Initialize PrettyTable for stockholders' equity
stockholders_equity_table = PrettyTable()

# Define field names for the table
field_names = ['Component'] + [extract_date(date) for date in date_string]
stockholders_equity_table.field_names = field_names

# Add data to the table row by row for each component
for component in stockholders_equity_components:
    data_row = [component] + [stockholders_equity_yearly[extract_date(date)].get(component, '') for date in date_string]
    stockholders_equity_table.add_row(data_row)
    
# Print the table
print("Stockholders' Equity:")
print(stockholders_equity_table)

Stockholders' Equity:
+--------------------------+-------------------+------------------+------------------+------------------+
|        Component         |     30/06/2023    |    30/06/2022    |    30/06/2021    |    30/06/2020    |
+--------------------------+-------------------+------------------+------------------+------------------+
|    Retained Earnings     | 118,848,000,000.0 | 84,281,000,000.0 | 57,055,000,000.0 | 34,566,000,000.0 |
|       Common Stock       |  93,718,000,000.0 | 86,939,000,000.0 | 83,111,000,000.0 | 80,552,000,000.0 |
|      Treasury Stock      |        None       |       None       |       None       |       None       |
| Other Equity Adjustments |  -6,343,000,000.0 | -4,678,000,000.0 | 1,822,000,000.0  | 3,186,000,000.0  |
+--------------------------+-------------------+------------------+------------------+------------------+


In [87]:
# Income Statement
income_statement = stock.income_stmt

date_string = list(income_statement.keys())

# Define field names for tables including dates
field_names = ['Component'] + [extract_date(date) for date in date_string]

# Initialize dictionaries to store data for each year
main_components_yearly = {extract_date(date): {} for date in date_string}

# Define the income components
main_components = ['Basic EPS', 'Diluted EPS', 'Basic Average Shares', 'Diluted Average Shares', 'EBIT', 'EBITDA','Normalized EBITDA', 
                   'Reconciled Cost Of Revenue', 'Reconciled Depreciation', 'Tax Rate For Calcs', 'Tax Effect Of Unusual Items']

# Iterate over the income statement data to extract main income components
for date in date_string:
    for component in main_components:
        try:
            main_components_yearly[extract_date(date)][component] = format_large_number(income_statement[pd.Timestamp(date)][component])
        except KeyError:
            main_components_yearly[extract_date(date)][component] = None

# PrettyTable for main income components
main_components_table = PrettyTable()
main_components_table.field_names = field_names

# Add data to the table for each main income component
for component in main_components:
    data_row = [component] + [main_components_yearly[date][component] for date in main_components_yearly]
    main_components_table.add_row(data_row)

# Print the main income table
print("\nMain Income Statement Components:")
print(main_components_table)


Main Income Statement Components:
+-----------------------------+-------------------+-------------------+--------------------+------------------+
|          Component          |     30/06/2023    |     30/06/2022    |     30/06/2021     |    30/06/2020    |
+-----------------------------+-------------------+-------------------+--------------------+------------------+
|          Basic EPS          |        9.72       |        9.7        |        8.12        |       5.82       |
|         Diluted EPS         |        9.68       |        9.65       |        8.05        |       5.76       |
|     Basic Average Shares    |  7,446,000,000.0  |  7,496,000,000.0  |  7,547,000,000.0   | 7,610,000,000.0  |
|    Diluted Average Shares   |  7,472,000,000.0  |  7,540,000,000.0  |  7,608,000,000.0   | 7,683,000,000.0  |
|             EBIT            |  91,279,000,000.0 |  85,779,000,000.0 |  73,448,000,000.0  | 55,627,000,000.0 |
|            EBITDA           | 105,140,000,000.0 | 100,239,000,000.0

In [88]:
date_string = list(income_statement.keys())

# Define field names for tables including dates
field_names = ['Component'] + [extract_date(date) for date in date_string]

# Initialize dictionary to store data for each year
revenue_yearly = {extract_date(date): {} for date in date_string}

# Define revenue components
revenue_components = ['Total Revenue', 'Operating Revenue', 'Cost Of Revenue', 'Gross Profit']

# Iterate over income statement data to extract revenue components
for date in date_string:
    for component in revenue_components:
        try:
            revenue_yearly[extract_date(date)][component] = format_large_number(income_statement[date][component])
        except KeyError:
            revenue_yearly[extract_date(date)][component] = None

# PrettyTable for revenue components
revenue_table = PrettyTable()
revenue_table.field_names = field_names

# Add data to the table for each component
for component in revenue_components:
    data_row = [component] + [revenue_yearly[date][component] for date in revenue_yearly]
    revenue_table.add_row(data_row)

# Print the table
print("Revenue:")
print(revenue_table)

Revenue:
+-------------------+-------------------+-------------------+-------------------+-------------------+
|     Component     |     30/06/2023    |     30/06/2022    |     30/06/2021    |     30/06/2020    |
+-------------------+-------------------+-------------------+-------------------+-------------------+
|   Total Revenue   | 211,915,000,000.0 | 198,270,000,000.0 | 168,088,000,000.0 | 143,015,000,000.0 |
| Operating Revenue | 211,915,000,000.0 | 198,270,000,000.0 | 168,088,000,000.0 | 143,015,000,000.0 |
|  Cost Of Revenue  |  65,863,000,000.0 |  62,650,000,000.0 |  52,232,000,000.0 |  46,078,000,000.0 |
|    Gross Profit   | 146,052,000,000.0 | 135,620,000,000.0 | 115,856,000,000.0 |  96,937,000,000.0 |
+-------------------+-------------------+-------------------+-------------------+-------------------+


In [89]:
date_string = list(income_statement.keys())

# Define field names for tables including dates
field_names = ['Component'] + [extract_date(date) for date in date_string]

# Initialize dictionaries to store data for each year
expenses_yearly = {extract_date(date): {} for date in date_string}

# Define the expense components
expense_components = ['Total Expenses', 'Operating Expense', 'Cost Of Revenue', 'Research And Development', 'Selling And Marketing Expense', 
                      'Selling General And Administration', 'Tax Provision']

# Iterate over the income statement data to extract expenses
for date in date_string:
    for component in expense_components:
        try:
            expenses_yearly[extract_date(date)][component] = format_large_number(income_statement[pd.Timestamp(date)][component])
        except KeyError:
            expenses_yearly[extract_date(date)][component] = None

# PrettyTable for expenses
expenses_table = PrettyTable()
expenses_table.field_names = field_names

# Add data to the table for each expense component
for component in expense_components:
    data_row = [component] + [expenses_yearly[date][component] for date in expenses_yearly]
    expenses_table.add_row(data_row)

# Print the expenses table
print("\nExpenses:")
print(expenses_table)


Expenses:
+------------------------------------+-------------------+-------------------+------------------+------------------+
|             Component              |     30/06/2023    |     30/06/2022    |    30/06/2021    |    30/06/2020    |
+------------------------------------+-------------------+-------------------+------------------+------------------+
|           Total Expenses           | 123,392,000,000.0 | 114,887,000,000.0 | 98,172,000,000.0 | 90,056,000,000.0 |
|         Operating Expense          |  57,529,000,000.0 |  52,237,000,000.0 | 45,940,000,000.0 | 43,978,000,000.0 |
|          Cost Of Revenue           |  65,863,000,000.0 |  62,650,000,000.0 | 52,232,000,000.0 | 46,078,000,000.0 |
|      Research And Development      |  27,195,000,000.0 |  24,512,000,000.0 | 20,716,000,000.0 | 19,269,000,000.0 |
|   Selling And Marketing Expense    |  22,759,000,000.0 |  21,825,000,000.0 | 20,117,000,000.0 | 19,598,000,000.0 |
| Selling General And Administration |  30,334,000,00

In [90]:
date_string = list(income_statement.keys())

# Define field names for tables including dates
field_names = ['Component'] + [extract_date(date) for date in date_string]

# Initialize dictionaries to store data for each year
income_yearly = {extract_date(date): {} for date in date_string}

# Define the income components
income_components = ['Net Income', 'Net Income Common Stockholders', 'Diluted NI Availto Com Stockholders', 'Operating Income', 
                     'Pretax Income','Gross Profit', 'Interest Income']

# Iterate over the income statement data to extract income components
for date in date_string:
    for component in income_components:
        try:
            income_yearly[extract_date(date)][component] = format_large_number(income_statement[pd.Timestamp(date)][component])
        except KeyError:
            income_yearly[extract_date(date)][component] = None

# PrettyTable for income components
income_table = PrettyTable()
income_table.field_names = field_names

# Add data to the table for each income component
for component in income_components:
    data_row = [component] + [income_yearly[date][component] for date in income_yearly]
    income_table.add_row(data_row)

# Print the income table
print("\nIncome:")
print(income_table)


Income:
+-------------------------------------+-------------------+-------------------+-------------------+------------------+
|              Component              |     30/06/2023    |     30/06/2022    |     30/06/2021    |    30/06/2020    |
+-------------------------------------+-------------------+-------------------+-------------------+------------------+
|              Net Income             |  72,361,000,000.0 |  72,738,000,000.0 |  61,271,000,000.0 | 44,281,000,000.0 |
|    Net Income Common Stockholders   |  72,361,000,000.0 |  72,738,000,000.0 |  61,271,000,000.0 | 44,281,000,000.0 |
| Diluted NI Availto Com Stockholders |  72,361,000,000.0 |  72,738,000,000.0 |  61,271,000,000.0 | 44,281,000,000.0 |
|           Operating Income          |  88,523,000,000.0 |  83,383,000,000.0 |  69,916,000,000.0 | 52,959,000,000.0 |
|            Pretax Income            |  89,311,000,000.0 |  83,716,000,000.0 |  71,102,000,000.0 | 53,036,000,000.0 |
|             Gross Profit            |

In [91]:
# Cashflow Statement
cash_flow = stock.cashflow

# Extract list of dates from the cash flow data
date_string = list(cash_flow.keys())

# Define field names for the table including dates
field_names = ['Component'] + [extract_date(date) for date in date_string]

# Initialize dictionary to store data for each year
main_components_yearly = {extract_date(date): {} for date in date_string}

# Define the main cash flow components
main_components = ['Capital Expenditure', 'Issuance Of Capital Stock', 'Issuance Of Debt', 'Repayment Of Debt', 
                   'Repurchase Of Capital Stock', 'Income Tax Paid Supplemental Data', 'Interest Paid Supplemental Data', 'Free Cash Flow']

# Iterate over cash flow data to extract main components for each year
for date in date_string:
    for component in main_components:
        try:
            # Try to extract the component value for the current date
            main_components_yearly[extract_date(date)][component] = format_large_number(cash_flow[pd.Timestamp(date)][component])
        except KeyError:
            # If the component is not available for the current date, assign None
            main_components_yearly[extract_date(date)][component] = None

# Create a PrettyTable instance with field names
main_components_table = PrettyTable()
main_components_table.field_names = field_names

# Add data to the table for each main cash flow component
for component in main_components:
    # Create a row with the component name and its values for each date
    data_row = [component] + [main_components_yearly[date][component] for date in main_components_yearly]
    # Add the row to the table
    main_components_table.add_row(data_row)

# Print the table
print("Main Cash Flow Statement Components: ")
print(main_components_table)

Main Cash Flow Statement Components: 
+-----------------------------------+-------------------+-------------------+-------------------+-------------------+
|             Component             |     30/06/2023    |     30/06/2022    |     30/06/2021    |     30/06/2020    |
+-----------------------------------+-------------------+-------------------+-------------------+-------------------+
|        Capital Expenditure        | -28,107,000,000.0 | -23,886,000,000.0 | -20,622,000,000.0 | -15,441,000,000.0 |
|     Issuance Of Capital Stock     |  1,866,000,000.0  |  1,841,000,000.0  |  1,693,000,000.0  |  1,343,000,000.0  |
|          Issuance Of Debt         |        nan        |        nan        |        nan        |        0.0        |
|         Repayment Of Debt         |  -2,750,000,000.0 |  -9,023,000,000.0 |  -3,750,000,000.0 |  -5,518,000,000.0 |
|    Repurchase Of Capital Stock    | -22,245,000,000.0 | -32,696,000,000.0 | -27,385,000,000.0 | -22,968,000,000.0 |
| Income Tax Paid 

In [92]:
# Extract list of dates from the cash flow data
date_string = list(cash_flow.keys())

# Define field names for the table including dates
field_names = ['Component'] + [extract_date(date) for date in date_string]

# Initialize dictionary to store data for each year
cfo_yearly = {extract_date(date): {} for date in date_string}

# Define the cash flow components
cfo_components = ['Operating Cash Flow', 'Change In Working Capital', 'Change In Other Working Capital', 'Stock Based Compensation', 
                  'Depreciation And Amortization', 'Depreciation Amortization Depletion', 'Deferred Tax', 'Net Income From Continuing Operations', 'Changes In Account Receivables']

# Iterate over cash flow data to extract components for each year
for date in date_string:
    for component in cfo_components:
        try:
            # Try to extract the component value for the current date
            cfo_yearly[extract_date(date)][component] = cash_flow[pd.Timestamp(date)][component]
        except KeyError:
            # If the component is not available for the current date, assign None
            cfo_yearly[extract_date(date)][component] = None

# Create a PrettyTable instance with field names
cfo_table = PrettyTable()
cfo_table.field_names = field_names

# Add data to the table for each cash flow component
for component in cfo_components:
    # Create a row with the component name and its values for each date
    data_row = [component] + [cfo_yearly[date][component] for date in cfo_yearly]
    # Add the row to the table
    cfo_table.add_row(data_row)

# Print the table
print("Cash Flow From Operations (CFO):")
print(cfo_table)

Cash Flow From Operations (CFO):
+---------------------------------------+---------------+---------------+---------------+---------------+
|               Component               |   30/06/2023  |   30/06/2022  |   30/06/2021  |   30/06/2020  |
+---------------------------------------+---------------+---------------+---------------+---------------+
|          Operating Cash Flow          | 87582000000.0 | 89035000000.0 | 76740000000.0 | 60675000000.0 |
|       Change In Working Capital       | -2388000000.0 |  446000000.0  |  -936000000.0 | -1483000000.0 |
|    Change In Other Working Capital    |  5177000000.0 |  5805000000.0 |  2324000000.0 | -1419000000.0 |
|        Stock Based Compensation       |  9611000000.0 |  7502000000.0 |  6118000000.0 |  5289000000.0 |
|     Depreciation And Amortization     | 13861000000.0 | 14460000000.0 | 11686000000.0 | 12796000000.0 |
|  Depreciation Amortization Depletion  | 13861000000.0 | 14460000000.0 | 11686000000.0 | 12796000000.0 |
|            

In [93]:
# Extract list of dates from the cash flow data
date_string = list(cash_flow.keys())

# Define field names for tables including dates
field_names = ['Component'] + [extract_date(date) for date in date_string]

# Initialize dictionary to store data for each year
cfi_yearly = {extract_date(date): {} for date in date_string}

# Define the cash flow from investing (CFI) components
cfi_components = ['Net PPE Purchase And Sale', 'Net Business Purchase And Sale', 'Net Investment Purchase And Sale', 'Purchase Of Investment', 'Sale Of Investment']

# Iterate over cash flow data to extract CFI components for each year
for date in date_string:
    for component in cfi_components:
        try:
            # Try to extract the component value for the current date
            cfi_yearly[extract_date(date)][component] = format_large_number(cash_flow[pd.Timestamp(date)][component])
        except KeyError:
            # If the component is not available for the current date, assign None
            cfi_yearly[extract_date(date)][component] = None

# Create a PrettyTable instance with field names
cfi_table = PrettyTable()
cfi_table.field_names = field_names

# Add data to the table for each CFI component
for component in cfi_components:
    # Create a row with the component name and its values for each date
    data_row = [component] + [cfi_yearly[date][component] for date in cfi_yearly]
    # Add the row to the table
    cfi_table.add_row(data_row)

# Print the table
print("Cash Flow From Investing (CFI): ")
print(cfi_table)

Cash Flow From Investing (CFI): 
+----------------------------------+-------------------+-------------------+-------------------+-------------------+
|            Component             |     30/06/2023    |     30/06/2022    |     30/06/2021    |     30/06/2020    |
+----------------------------------+-------------------+-------------------+-------------------+-------------------+
|    Net PPE Purchase And Sale     | -28,107,000,000.0 | -23,886,000,000.0 | -20,622,000,000.0 | -15,441,000,000.0 |
|  Net Business Purchase And Sale  |  -1,670,000,000.0 | -22,038,000,000.0 |  -8,909,000,000.0 |  -2,521,000,000.0 |
| Net Investment Purchase And Sale |  10,213,000,000.0 |  18,438,000,000.0 |  2,876,000,000.0  |  6,980,000,000.0  |
|      Purchase Of Investment      | -37,651,000,000.0 | -26,456,000,000.0 | -62,924,000,000.0 | -77,190,000,000.0 |
|        Sale Of Investment        |  47,864,000,000.0 |  44,894,000,000.0 |  65,800,000,000.0 |  84,170,000,000.0 |
+------------------------------

In [94]:
# Extract list of dates from the cash flow data
date_string = list(cash_flow.keys())

# Define field names for tables including dates
field_names = ['Component'] + [extract_date(date) for date in date_string]

# Initialize dictionary to store data for each year
cff_yearly = {extract_date(date): {} for date in date_string}

# Define the cash flow from financing (CFF) components
cff_components = ['Net Long Term Debt Issuance', 'Long Term Debt Issuance', 'Long Term Debt Payments', 'Net Common Stock Issuance', 'Common Stock Issuance', 
                  'Common Stock Payments', 'Net Other Financing Charges']

# Iterate over cash flow data to extract CFF components for each year
for date in date_string:
    for component in cff_components:
        try:
            # Try to extract the component value for the current date
            cff_yearly[extract_date(date)][component] = format_large_number(cash_flow[pd.Timestamp(date)][component])
        except KeyError:
            # If the component is not available for the current date, assign None
            cff_yearly[extract_date(date)][component] = None

# Create a PrettyTable instance with field names
cff_table = PrettyTable()
cff_table.field_names = field_names

# Add data to the table for each CFF component
for component in cff_components:
    # Create a row with the component name and its values for each date
    data_row = [component] + [cff_yearly[date][component] for date in cff_yearly]
    # Add the row to the table
    cff_table.add_row(data_row)

# Print the table
print("Cash Flow From Financing (CFF): ")
print(cff_table)

Cash Flow From Financing (CFF): 
+-----------------------------+-------------------+-------------------+-------------------+-------------------+
|          Component          |     30/06/2023    |     30/06/2022    |     30/06/2021    |     30/06/2020    |
+-----------------------------+-------------------+-------------------+-------------------+-------------------+
| Net Long Term Debt Issuance |  -2,750,000,000.0 |  -9,023,000,000.0 |  -3,750,000,000.0 |  -5,518,000,000.0 |
|   Long Term Debt Issuance   |        nan        |        nan        |        nan        |        0.0        |
|   Long Term Debt Payments   |  -2,750,000,000.0 |  -9,023,000,000.0 |  -3,750,000,000.0 |  -5,518,000,000.0 |
|  Net Common Stock Issuance  | -20,379,000,000.0 | -30,855,000,000.0 | -25,692,000,000.0 | -21,625,000,000.0 |
|    Common Stock Issuance    |  1,866,000,000.0  |  1,841,000,000.0  |  1,693,000,000.0  |  1,343,000,000.0  |
|    Common Stock Payments    | -22,245,000,000.0 | -32,696,000,000.0 |

In [95]:
# Extract list of dates from the cash flow data
date_string = list(cash_flow.keys())

# Define field names for tables including dates
field_names = ['Component'] + [extract_date(date) for date in date_string]

# Initialize dictionaries to store data for each year
ecp_yearly = {extract_date(date): {} for date in date_string}

# Define the cash flow components for End Cash Position (ECP)
ecp_components = ['Changes In Cash', 'Effect Of Exchange Rate Changes', 'Beginning Cash Position']

# Iterate over cash flow data to extract ECP components for each year
for date in date_string:
    for component in ecp_components:
        try:
            ecp_yearly[extract_date(date)][component] = format_large_number(cash_flow[pd.Timestamp(date)][component])
        except KeyError:
            ecp_yearly[extract_date(date)][component] = None

# Create a PrettyTable instance with field names
ecp_table = PrettyTable()
ecp_table.field_names = field_names

# Add data to the table for each ECP component
for component in ecp_components:
    data_row = [component] + [ecp_yearly[date][component] for date in ecp_yearly]
    ecp_table.add_row(data_row)

# Print the table
print("End Cash Position (ECP): ")
print(ecp_table)

End Cash Position (ECP): 
+---------------------------------+------------------+------------------+------------------+------------------+
|            Component            |    30/06/2023    |    30/06/2022    |    30/06/2021    |    30/06/2020    |
+---------------------------------+------------------+------------------+------------------+------------------+
|         Changes In Cash         | 20,967,000,000.0 |  -152,000,000.0  |  677,000,000.0   | 2,421,000,000.0  |
| Effect Of Exchange Rate Changes |  -194,000,000.0  |  -141,000,000.0  |  -29,000,000.0   |  -201,000,000.0  |
|     Beginning Cash Position     | 13,931,000,000.0 | 14,224,000,000.0 | 13,576,000,000.0 | 11,356,000,000.0 |
+---------------------------------+------------------+------------------+------------------+------------------+


In [96]:
# Holders
major_holders = stock.major_holders

print(major_holders)
# Define the left side of the major_holders_table
breakdown_side = [
    "Percentage of Shares Held by All Insider",
    "Percentage of Shares Held by Institutions",
    "Percentage of Float Held by Institutions",
    "Number of Institutions Holding Shares"
]

# Extract holders and percentage held
holders = [holder for holder in major_holders.index]

# List to store percentages
percentages = []

# Counter to keep track of the row number
counter = 0

# Iterate over major holders data
for index, row in major_holders.iterrows():
    breakdown = index
    value = row['Value']
    
    # Check if we're within the first 3 rows
    if counter < 3:
        value *= 100  # Multiply value by 100
    percentages.append(value)
    
    # Increment counter
    counter += 1

# Create a PrettyTable object
major_holders_table = PrettyTable()
major_holders_table.field_names = ["Breakdown", "Value"]

# Add data to the major_holders_table
for left, percentage in zip(breakdown_side, percentages):
    major_holders_table.add_row([left, percentage])

print("Major Holders:")
print(major_holders_table)

Breakdown                          Value
insidersPercentHeld              0.00054
institutionsPercentHeld          0.73648
institutionsFloatPercentHeld     0.73688
institutionsCount             6833.00000
Major Holders:
+-------------------------------------------+-------------+
|                 Breakdown                 |    Value    |
+-------------------------------------------+-------------+
|  Percentage of Shares Held by All Insider | 0.054000004 |
| Percentage of Shares Held by Institutions |    73.648   |
|  Percentage of Float Held by Institutions |  73.688006  |
|   Number of Institutions Holding Shares   |    6833.0   |
+-------------------------------------------+-------------+


In [97]:
# Top Institutional Holders
institutional_holders = stock.institutional_holders

# Create a PrettyTable object
institutional_holders_table = PrettyTable()
institutional_holders_table.field_names = ["Holder", "Shares Held", "Date Reported", "Percentage Held"]


# Add data to the institutional_holders_table
for index, row in institutional_holders.iterrows():
    holder = row['Holder']
    shares_held = row['Shares']
    date_reported = extract_date(row['Date Reported']) 
    percentage = '{:.2f}'.format(row['pctHeld'] * 100)
    institutional_holders_table.add_row([holder, shares_held, date_reported, percentage])

# Print the institutional_holders_table
print(institutional_holders_table)

+-----------------------------------+-------------+---------------+-----------------+
|               Holder              | Shares Held | Date Reported | Percentage Held |
+-----------------------------------+-------------+---------------+-----------------+
|         Vanguard Group Inc        |  653247546  |   30/06/2023  |       8.79      |
|           Blackrock Inc.          |  536245027  |   30/06/2023  |       7.22      |
|      State Street Corporation     |  293614078  |   30/06/2023  |       3.95      |
|              FMR, LLC             |  210111250  |   30/06/2023  |       2.83      |
|   Price (T.Rowe) Associates Inc   |  159904219  |   30/06/2023  |       2.15      |
|   Geode Capital Management, LLC   |  148129470  |   30/06/2023  |       1.99      |
|           Morgan Stanley          |  123498322  |   30/06/2023  |       1.66      |
|     JP Morgan Chase & Company     |  113024837  |   30/06/2023  |       1.52      |
| Norges Bank Investment Management |   86316926  |   

In [98]:
# Top Mutual Fund Holders
mutualfund_holders = stock.mutualfund_holders

# Create a PrettyTable object
mutualfund_holders_table = PrettyTable()
mutualfund_holders_table.field_names = ["Holder", "Shares Held", "Date Reported", "Percentage Held"]

# Add data to the mutualfund_holders_table
for index, row in mutualfund_holders.iterrows():
    # Extract data for each row
    holder = row['Holder']  # Extract the holder name
    shares_held = row['Shares']  # Extract the number of shares held
    date_reported = extract_date(row['Date Reported'])  # Format the date reported using the extract_date function
    percentage = '{:.2f}'.format(row['pctHeld'] * 100)  # Format the percentage held to two decimal places
    
    # Add the extracted data as a row to the mutualfund_holders_table
    mutualfund_holders_table.add_row([holder, shares_held, date_reported, percentage])

# Print the mutualfund_holders_table
print(mutualfund_holders_table)

+------------------------------------------------------------+-------------+---------------+-----------------+
|                           Holder                           | Shares Held | Date Reported | Percentage Held |
+------------------------------------------------------------+-------------+---------------+-----------------+
|           Vanguard Total Stock Market Index Fund           |  231883087  |   30/06/2023  |       3.12      |
|                  Vanguard 500 Index Fund                   |  177034936  |   30/06/2023  |       2.38      |
|                  Fidelity 500 Index Fund                   |   85181943  |   31/08/2023  |       1.15      |
|                   SPDR S&P 500 ETF Trust                   |   83514381  |   30/09/2023  |       1.12      |
|                  iShares Core S&P 500 ETF                  |   70264910  |   30/09/2023  |       0.95      |
|                 Vanguard Growth Index Fund                 |   63262454  |   30/06/2023  |       0.85      |
|

In [99]:
def get_ticker_name(ticker_symbol):
    try:
        ticker = yf.Ticker(ticker_symbol)
        
        long_name = ticker.info.get('longName')
        if long_name:
            return long_name
        
        short_name = ticker.info.get('shortName')
        if short_name:
            return short_name
                
        return ticker_symbol
    except Exception as e:
        print(f"Error occurred while fetching ticker name for {ticker_symbol}: {e}")
        return None

In [100]:
# Peers

def fetch_news(choice, api_key):
    url = f"https://api.polygon.io/v2/reference/news?ticker={choice}&apiKey={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch news:", response.text)
        return None

api_key = "V2jiIqWR6ZKdcV0BC_sh84w6ZuaxArdE"
news_data1 = fetch_news(choice, api_key)

tickers_list = []

if news_data1:
    for article in news_data1['results']:
        tickers_list.extend(article['tickers'])

news_data2 = stock.news

related_tickers = []

if news_data2:
    for article in news_data2:
        related_tickers.extend(article.get('relatedTickers', []))

related_tickers = list(set(related_tickers))
tickers_list = list(set(tickers_list))
combined_tickers = list(set(related_tickers + tickers_list))

if choice in combined_tickers:
    combined_tickers.remove(choice)
else:
    print(f"Warning: {choice} not found in combined_tickers")
combined_tickers = sorted(combined_tickers)

def get_ticker_info(ticker):
    try:
        ticker_info = yf.Ticker(ticker)
        
        return {
            'Ticker': ticker,
            'Name': ticker_info.info.get('longName'),
            'Sector': ticker_info.info.get('sector'),
            'Industry': ticker_info.info.get('industry'),
            'Market Capitalization': format_large_number("{:.2f}".format(ticker_info.info.get('marketCap'))) if ticker_info.info.get('marketCap') is not None else None,
            'Type': ticker_info.info.get('quoteType')
        }
    except Exception as e:
        print(f"Error occurred while fetching data for {ticker}: {e}")
        return None

def print_peer_info(choice, combined_tickers):
    table = PrettyTable()
    table.field_names = ['Ticker', 'Name', 'Sector', 'Industry', 'Market Capitalization']
    
    choice_info = get_ticker_info(choice)
    if choice_info: 
        table.add_row([choice_info['Ticker'], choice_info['Name'], choice_info['Sector'], choice_info['Industry'], choice_info['Market Capitalization']])
        
    filtered_tickers_list = []
    for ticker in combined_tickers:
        ticker_info = get_ticker_info(ticker)
        if ticker_info['Type'] == 'EQUITY' and (ticker_info['Sector'] == stock.info['sector'] or ticker_info['Industry'] == stock.info['industry']): 
            table.add_row([ticker_info['Ticker'], ticker_info['Name'], ticker_info['Sector'], ticker_info['Industry'], ticker_info['Market Capitalization']])
            filtered_tickers_list.append(ticker)
    
    print(table)
    return filtered_tickers_list

combined_tickers = print_peer_info(choice, combined_tickers)
print(choice)
print(combined_tickers)
print(related_tickers)
print(tickers_list)

+--------+----------------------------------------------------+------------+-------------------------------------+-----------------------+
| Ticker |                        Name                        |   Sector   |               Industry              | Market Capitalization |
+--------+----------------------------------------------------+------------+-------------------------------------+-----------------------+
|  MSFT  |               Microsoft Corporation                | Technology |      Software - Infrastructure      |  3,117,876,248,576.00 |
|  AAPL  |                     Apple Inc.                     | Technology |         Consumer Electronics        |  2,909,798,662,144.00 |
|  AMAT  |              Applied Materials, Inc.               | Technology | Semiconductor Equipment & Materials |   175,917,514,752.00  |
|  AMD   |            Advanced Micro Devices, Inc.            | Technology |            Semiconductors           |   264,609,497,088.00  |
|  CFLT  |                 

In [101]:
# Valuation Metrics
def get_valuation_metrics(ticker):
    try:
        ticker_info = yf.Ticker(ticker)
        
        return {
            'Ticker': ticker,
            'Trailing P/E Ratio': "{:.2f}".format(ticker_info.info.get('trailingPE')) if ticker_info.info.get('trailingPE') is not None else None,
            'P/S Ratio': "{:.2f}".format(ticker_info.info.get('priceToSalesTrailing12Months')) if ticker_info.info.get('priceToSalesTrailing12Months') is not None else None,
            'EV/EBITDA Ratio': "{:.2f}".format(ticker_info.info.get('enterpriseToEbitda')) if ticker_info.info.get('enterpriseToEbitda') is not None else None,
            'Trailing PEG Ratio': "{:.2f}".format(ticker_info.info.get('trailingPegRatio')) if ticker_info.info.get('trailingPegRatio') is not None else None,
            'Dividend Yield': "{:.2f}".format(ticker_info.info.get('dividendYield')) if ticker_info.info.get('dividendYield') is not None else None,
        }

    except Exception as e:
        print(f"Error occurred while fetching data for {ticker}: {e}")
        return None

def print_valuation_metrics(choice, tickers_list):
    # Create a PrettyTable object with headers
    table = PrettyTable()
    table.field_names = ['Ticker', 'Trailing P/E Ratio', 'P/S Ratio', 'EV/EBITDA Ratio', 'Trailing PEG Ratio', 'Dividend Yield (%)']
    
    # Add row for the choice's valuation metrics
    choice_metrics = get_valuation_metrics(choice)
    if choice_metrics:
        table.add_row([get_ticker_name(choice), choice_metrics['Trailing P/E Ratio'], choice_metrics['P/S Ratio'], choice_metrics['EV/EBITDA Ratio'], 
                       choice_metrics['Trailing PEG Ratio'], choice_metrics['Dividend Yield']])
    
    # Add rows for each peer's valuation metrics
    for ticker in tickers_list:
        peer_metrics = get_valuation_metrics(ticker)
        if peer_metrics:
            table.add_row([get_ticker_name(ticker), peer_metrics['Trailing P/E Ratio'], peer_metrics['P/S Ratio'], peer_metrics['EV/EBITDA Ratio'], 
                           peer_metrics['Trailing PEG Ratio'], peer_metrics['Dividend Yield']])
    
    # Print the table
    print("Valuation Metrics:")
    print(table)

print_valuation_metrics(choice, combined_tickers)

Valuation Metrics:
+----------------------------------------------------+--------------------+-----------+-----------------+--------------------+--------------------+
|                       Ticker                       | Trailing P/E Ratio | P/S Ratio | EV/EBITDA Ratio | Trailing PEG Ratio | Dividend Yield (%) |
+----------------------------------------------------+--------------------+-----------+-----------------+--------------------+--------------------+
|               Microsoft Corporation                |       36.35        |   13.18   |      25.20      |        2.08        |        0.01        |
|                     Apple Inc.                     |       29.47        |    7.62   |      22.75      |        2.25        |        0.01        |
|              Applied Materials, Inc.               |       24.91        |    6.64   |      21.67      |        2.56        |        0.01        |
|            Advanced Micro Devices, Inc.            |       240.75       |   11.61   |      

In [102]:
# Profitability Metrics
def get_profitability_metrics(ticker):
    try:
        ticker_info = yf.Ticker(ticker)
        financials = ticker_info.financials
        balance_sheet = ticker_info.balance_sheet
        
        return {
            'Ticker': ticker,
            'ROE': "{:.2f}".format(ticker_info.info.get('returnOnEquity') * 100) if ticker_info.info.get('returnOnEquity') is not None else None,
            'ROA': "{:.2f}".format(ticker_info.info.get('returnOnAssets') * 100) if ticker_info.info.get('returnOnAssets') is not None else None,
            'Gross Margin': "{:.2f}".format(ticker_info.info.get('grossMargins') * 100) if ticker_info.info.get('grossMargins') is not None else None,
            'Operating Margin': "{:.2f}".format(ticker_info.info.get('operatingMargins') * 100)if ticker_info.info.get('operatingMargins') is not None else None,
            'Profit Margin': "{:.2f}".format(ticker_info.info.get('profitMargins') * 100) if ticker_info.info.get('profitMargins') is not None else None,
        }

    except Exception as e:
        print(f"Error occurred while fetching data for {ticker}: {e}")
        return None

def print_profitability_metrics(choice, tickers_list):
    # Create a PrettyTable object with headers
    table = PrettyTable()
    table.field_names = ['Ticker', 'ROE Ratio', 'ROA Ratio', 'Gross Margin (%)', 'Operating Margin (%)', 'Profit Margin (%)']

    # Add row for the choice's profitability metrics
    choice_metrics = get_profitability_metrics(choice)
    if choice_metrics:
        table.add_row([get_ticker_name(choice), choice_metrics['ROE'], choice_metrics['ROA'], 
                       choice_metrics['Gross Margin'], choice_metrics['Operating Margin'], 
                       choice_metrics['Profit Margin']])
    
    # Add rows for each peer's profitability metrics
    for ticker in tickers_list:
        peer_metrics = get_profitability_metrics(ticker)
        if peer_metrics:
            table.add_row([get_ticker_name(ticker), peer_metrics['ROE'], peer_metrics['ROA'], 
                           peer_metrics['Gross Margin'], peer_metrics['Operating Margin'], 
                           peer_metrics['Profit Margin']])
    
    # Print the table
    print("Profitability Metrics:")
    print(table)

print_profitability_metrics(choice, combined_tickers)

Profitability Metrics:
+----------------------------------------------------+-----------+-----------+------------------+----------------------+-------------------+
|                       Ticker                       | ROE Ratio | ROA Ratio | Gross Margin (%) | Operating Margin (%) | Profit Margin (%) |
+----------------------------------------------------+-----------+-----------+------------------+----------------------+-------------------+
|               Microsoft Corporation                |   38.49   |   15.29   |      69.89       |        44.59         |       36.43       |
|                     Apple Inc.                     |   147.25  |   22.07   |      45.59       |        30.74         |       26.31       |
|              Applied Materials, Inc.               |   45.19   |   15.67   |      47.18       |        28.77         |       27.57       |
|            Advanced Micro Devices, Inc.            |    2.01   |    0.54   |      50.56       |         0.66         |        4.9

In [103]:
# Leverage Metrics
def get_leverage_metrics(ticker):
    try:
        ticker_info = yf.Ticker(ticker)
        balance_sheet = ticker_info.balance_sheet
        
        return {
            'Ticker': ticker,
            'Debt-to-Equity Ratio': "{:.2f}".format(ticker_info.info.get('debtToEquity')) if ticker_info.info.get('debtToEquity') is not None else None
        }
    except Exception as e:
        print(f"Error occurred while fetching data for {ticker}: {e}")
        return None

def print_leverage_metrics(choice, tickers_list):
    # Create a PrettyTable object with headers
    table = PrettyTable()
    table.field_names = ['Ticker', 'Debt-to-Equity Ratio']
    
    # Add row for the choice's leverage metrics
    choice_metrics = get_leverage_metrics(choice)
    if choice_metrics:
        table.add_row([get_ticker_name(choice), choice_metrics['Debt-to-Equity Ratio']])
    
    # Add rows for each peer's leverage metrics
    for ticker in tickers_list:
        peer_metrics = get_leverage_metrics(ticker)
        if peer_metrics:
            table.add_row([get_ticker_name(ticker), peer_metrics['Debt-to-Equity Ratio']])
    
    # Print the table
    print("Leverage Metrics:")
    print(table)

print_leverage_metrics(choice, combined_tickers)

Leverage Metrics:
+----------------------------------------------------+----------------------+
|                       Ticker                       | Debt-to-Equity Ratio |
+----------------------------------------------------+----------------------+
|               Microsoft Corporation                |        41.96         |
|                     Apple Inc.                     |        140.97        |
|              Applied Materials, Inc.               |        30.56         |
|            Advanced Micro Devices, Inc.            |         5.33         |
|                  Confluent, Inc.                   |        131.64        |
|                  Salesforce, Inc.                  |        22.74         |
|             CrowdStrike Holdings, Inc.             |        33.93         |
|                   Datadog, Inc.                    |        43.48         |
|    International Business Machines Corporation     |        269.45        |
|                 NVIDIA Corporation          

In [104]:
# Risk Metrics
def get_risk_metrics(ticker):
    try:
        ticker_info = yf.Ticker(ticker)
        
        return {
            'Ticker': ticker,
            'Beta': "{:.2f}".format(ticker_info.info.get('beta')) if ticker_info.info.get('beta') is not None else None
        }
    except Exception as e:
        print(f"Error occurred while fetching data for {ticker}: {e}")
        return None

def print_leverage_metrics(choice, tickers_list):
    # Create a PrettyTable object with headers
    table = PrettyTable()
    table.field_names = ['Ticker', 'Beta']
    
    # Add row for the choice's leverage metrics
    choice_metrics = get_risk_metrics(choice)
    if choice_metrics:
        table.add_row([get_ticker_name(choice), choice_metrics['Beta']])
    
    # Add rows for each peer's leverage metrics
    for ticker in tickers_list:
        risk_metrics = get_risk_metrics(ticker)
        if risk_metrics:
            table.add_row([get_ticker_name(ticker), risk_metrics['Beta']])
    
    # Print the table
    print("Risk Metrics:")
    print(table)

print_leverage_metrics(choice, combined_tickers)

Risk Metrics:
+----------------------------------------------------+------+
|                       Ticker                       | Beta |
+----------------------------------------------------+------+
|               Microsoft Corporation                | 0.89 |
|                     Apple Inc.                     | 1.26 |
|              Applied Materials, Inc.               | None |
|            Advanced Micro Devices, Inc.            | 1.66 |
|                  Confluent, Inc.                   | 0.93 |
|                  Salesforce, Inc.                  | 1.30 |
|             CrowdStrike Holdings, Inc.             | 1.07 |
|                   Datadog, Inc.                    | 1.14 |
|    International Business Machines Corporation     | 0.74 |
|                 NVIDIA Corporation                 | 1.75 |
|                   Snowflake Inc.                   | 0.90 |
| Taiwan Semiconductor Manufacturing Company Limited | 1.24 |
|              Uber Technologies, Inc.               | 1

In [105]:
# Expected Annual Growth Rates
def get_expected_annual_earnings_growth(ticker):
    try:
        ticker_info = yf.Ticker(ticker)
        summary_info = ticker_info.info
    
        return {
                'Ticker': ticker,
                'Expected Annual Earnings Growth Rate': "{:.2f}".format(ticker_info.info.get('earningsGrowth')) if ticker_info.info.get('earningsGrowth') is not None else None,
                'Expected Annual Revenue Growth Rate': "{:.2f}".format(ticker_info.info.get('revenueGrowth')) if ticker_info.info.get('revenueGrowth') is not None else None
        }
  
    except Exception as e:
        print(f"Error occurred while fetching data for {ticker}: {e}")
        return None

def print_expected_annual_earnings_growth(choice, tickers_list):
    # Create a PrettyTable object with headers
    table = PrettyTable()
    table.field_names = ['Ticker', 'Expected Annual Earnings Growth Rate (%)', 'Expected Annual Revenue Growth Rate (%)']
    
    # Add row for the choice's expected annual earnings growth rate
    choice_growth_rate = get_expected_annual_earnings_growth(choice)
    if choice_growth_rate:
        table.add_row([get_ticker_name(choice), choice_growth_rate['Expected Annual Earnings Growth Rate'], choice_growth_rate['Expected Annual Revenue Growth Rate']])
    
    # Add rows for each peer's expected annual earnings growth rate
    for ticker in tickers_list:
        peer_growth_rate = get_expected_annual_earnings_growth(ticker)
        if peer_growth_rate:
            table.add_row([get_ticker_name(ticker), peer_growth_rate['Expected Annual Earnings Growth Rate'],  peer_growth_rate['Expected Annual Revenue Growth Rate']])
    
    # Print the table
    print("Expected Annual Growth Rates:")
    print(table)

print_expected_annual_earnings_growth(choice, combined_tickers)

Expected Annual Growth Rates:
+----------------------------------------------------+------------------------------------------+-----------------------------------------+
|                       Ticker                       | Expected Annual Earnings Growth Rate (%) | Expected Annual Revenue Growth Rate (%) |
+----------------------------------------------------+------------------------------------------+-----------------------------------------+
|               Microsoft Corporation                |                   0.20                   |                   0.17                  |
|                     Apple Inc.                     |                   0.01                   |                  -0.04                  |
|              Applied Materials, Inc.               |                   0.11                   |                   0.00                  |
|            Advanced Micro Devices, Inc.            |                   None                   |                   0.02          

In [106]:
# Indices

# Read the HTML content and extract the first table
df_list = pd.read_html('https://finance.yahoo.com/world-indices/')
majorStockIdx = df_list[0]

# Extract the time zone from the Ticker object
timezone = stock.info.get('timeZoneFullName')
stock_country = timezone.split('/')[0].strip()

# Create a dictionary to store symbols sorted by country
symbols_by_country = {}

# Iterate through each row of the DataFrame
for index, row in majorStockIdx.iterrows():
    # Extract the ticker symbol from the "Symbol" column
    symbol = row['Symbol']
    
    # Create a Ticker object for the symbol
    ticker = yf.Ticker(symbol)
    
    # Extract the time zone from the Ticker object
    timezone = ticker.info.get('timeZoneFullName')
    
    # Extract the country name from the time zone string
    country = timezone.split('/')[0].strip()
    
    # Combine "Australia" and "Pacific" into one category
    if country == "Australia" or country == "Pacific":
        country = "Australia/Pacific"
    
    # Add the symbol to the list for the corresponding country
    symbols_by_country.setdefault(country, []).append(symbol)

for country, symbols in symbols_by_country.items():
    print(f"{country}: {symbols}")

America: ['^GSPC', '^DJI', '^IXIC', '^NYA', '^XAX', '^RUT', '^VIX', '^GSPTSE', '^BVSP', '^MXX', '^IPSA', '^MERV']
Europe: ['^BUK100P', '^FTSE', '^GDAXI', '^FCHI', '^STOXX50E', '^N100', '^BFX', 'IMOEX.ME']
Asia: ['^N225', '^HSI', '000001.SS', '399001.SZ', '^STI', '^BSESN', '^JKSE', '^KLSE', '^KS11', '^TWII', '^TA125.TA']
Australia/Pacific: ['^AXJO', '^AORD', '^NZ50']
Africa: ['^CASE30', '^JN0U.JO']


In [107]:
def get_index_info(stock_country_symbols):
    table = PrettyTable()
    table.field_names = ["Index", "Name", "Currency", "Exchange", " 52 Week Range"]
    
    for symbol in stock_country_symbols:
        ticker = yf.Ticker(symbol)
        index = symbol
        name = get_ticker_name(symbol)
        currency = ticker.info.get('currency', 'None')
        exchange = ticker.info.get('exchange', 'None')
        week_range = str(format_large_number(ticker.info.get('fiftyTwoWeekLow', 0))) + " - " + str(format_large_number(ticker.info.get('fiftyTwoWeekHigh', 0)))
        table.add_row([index, name, currency, exchange, week_range])
    return table

# Get symbols for the country of the stock
stock_country_symbols = symbols_by_country.get(stock_country, [])

# Fetch information for the indices
index_info_table = get_index_info(stock_country_symbols)

# Print the table
print(index_info_table)

+---------+------------------------------+----------+----------+-----------------------+
|  Index  |             Name             | Currency | Exchange |      52 Week Range    |
+---------+------------------------------+----------+----------+-----------------------+
|  ^GSPC  |           S&P 500            |   USD    |   SNP    |  4,103.78 - 5,325.49  |
|   ^DJI  | Dow Jones Industrial Average |   USD    |   DJI    |  32,327.2 - 40,051.05 |
|  ^IXIC  |       NASDAQ Composite       |   USD    |   NIM    | 12,415.85 - 16,797.83 |
|   ^NYA  |     NYSE COMPOSITE (DJ)      |   USD    |   NYS    | 14,638.37 - 18,389.78 |
|   ^XAX  |  NYSE AMEX COMPOSITE INDEX   |   USD    |   NYS    |  3,824.21 - 5,094.94  |
|   ^RUT  |         Russell 2000         |   USD    |   WCB    |  1,633.67 - 2,135.46  |
|   ^VIX  |    CBOE Volatility Index     |   USD    |   CXI    |     12.14 - 35.05     |
| ^GSPTSE |   S&P/TSX Composite index    |   CAD    |   TSI    |  18,692.1 - 22,470.3  |
|  ^BVSP  |          

In [108]:
def calculate_performance_metrics(stock_ticker):
    # Calculate start date as one year ago from today
    start_date = (datetime.datetime.now() - datetime.timedelta(days=365)).strftime('%Y-%m-%d')
    
    # Get today's date
    end_date = datetime.datetime.now().strftime('%Y-%m-%d')
    
    # Download historical price data for the stock
    stock_data = yf.download(stock_ticker, start=start_date, end=end_date)
    
    # Check if stock_data is empty
    if stock_data.empty:
        print(f"No data available for {stock_ticker} within the specified time frame.")
        return None
    
    # Calculate daily returns for the stock
    stock_returns = stock_data['Adj Close'].pct_change().dropna()
    
    # Calculate total return
    total_return = (stock_data['Adj Close'][-1] / stock_data['Adj Close'][0]) - 1
    
    # Calculate annualized return
    annualized_return = ((1 + total_return) ** (1 / len(stock_returns))) - 1
    
    # Calculate volatility (standard deviation)
    volatility = stock_returns.std()
    
    # Calculate Maximum Drawdown
    max_drawdown = (stock_data['Adj Close'].min() / stock_data['Adj Close'].max()) - 1
    
    return {
        'Total Return': "{:.2f}".format(total_return),
        'Annualized Return': "{:.2f}".format(annualized_return),
        'Volatility': "{:.2f}".format(volatility),
        'Maximum Drawdown': "{:.2f}".format(max_drawdown)
    }

metrics_table = PrettyTable()
metrics_table.field_names = ['Name', 'Total Return', 'Annualized Return', 'Volatility', 'Maximum Drawdown']

choice_metrics = calculate_performance_metrics(choice)
if choice_metrics:
    metrics_table.add_row([get_ticker_name(choice),  choice_metrics['Total Return'], choice_metrics['Annualized Return'], choice_metrics['Volatility'], choice_metrics['Maximum Drawdown']])

for symbol in stock_country_symbols:
    index_metrics = calculate_performance_metrics(symbol)
    if index_metrics:
        metrics_table.add_row([get_ticker_name(symbol), index_metrics['Total Return'], index_metrics['Annualized Return'], index_metrics['Volatility'], index_metrics['Maximum Drawdown']])
    else:
        print(f"No metrics available for {symbol}")
        stock_country_symbols.remove(symbol)

print("Performance Metrics:")
print(metrics_table)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['^IPSA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-05-18 -> 2024-05-17)')


No data available for ^IPSA within the specified time frame.
No metrics available for ^IPSA
Performance Metrics:
+------------------------------+--------------+-------------------+------------+------------------+
|             Name             | Total Return | Annualized Return | Volatility | Maximum Drawdown |
+------------------------------+--------------+-------------------+------------+------------------+
|    Microsoft Corporation     |     0.33     |        0.00       |    0.01    |      -0.28       |
|           S&P 500            |     0.26     |        0.00       |    0.01    |      -0.22       |
| Dow Jones Industrial Average |     0.19     |        0.00       |    0.01    |      -0.19       |
|       NASDAQ Composite       |     0.32     |        0.00       |    0.01    |      -0.25       |
|     NYSE COMPOSITE (DJ)      |     0.19     |        0.00       |    0.01    |      -0.20       |
|  NYSE AMEX COMPOSITE INDEX   |     0.25     |        0.00       |    0.01    |      -

In [109]:
def calculate_market_metrics(stock_ticker, index_ticker):
    # Calculate start date as one year ago from today
    start_date = (datetime.datetime.now() - datetime.timedelta(days=365)).strftime('%Y-%m-%d')
    
    # Get today's date
    end_date = datetime.datetime.now().strftime('%Y-%m-%d')
    
    # Download historical price data for the stock and the index
    stock_data = yf.download(stock_ticker, start=start_date, end=end_date)
    index_data = yf.download(index_ticker, start=start_date, end=end_date)
    
    # Check if data is available
    if stock_data.empty or index_data.empty:
        print("No data available for the specified tickers within the specified time frame.")
        return None
    
    # Calculate market capitalization of the index
    index_market_cap = index_data['Adj Close'][-1] * index_data['Volume'][-1]
    
    # Calculate beta
    stock_returns = stock_data['Adj Close'].pct_change().dropna()
    index_returns = index_data['Adj Close'].pct_change().dropna()
    beta = stock_returns.cov(index_returns) / index_returns.var()
    
    # Calculate correlation with index
    stock_returns, index_returns = stock_returns.align(index_returns, join="inner")
    correlation = stock_returns.corr(index_returns)
    
    # Calculate liquidity (average daily volume)
    index_avg_volume = index_data['Volume'].mean()
    
    # Calculate short interest ratio
    # Note: Short interest data may not be available from Yahoo Finance directly
    
    return {
        'Market Capitalization': format_large_number("{:.2f}".format(index_market_cap)),
        'Beta': "{:.2f}".format(beta),
        'Correlation': "{:.2f}".format(correlation),
        'Liquidity (Avg. Daily Volume)': format_large_number("{:.2f}".format(index_avg_volume))
    }

market_table = PrettyTable()
market_table.field_names = ['Index', 'Market Capitalization', 'Beta', 'Correlation', 'Liquidity (Avg. Daily Volume)']
    
for symbol in stock_country_symbols:
    market_metrics = calculate_market_metrics(choice, symbol)

    if market_metrics:
        market_table.add_row([
            get_ticker_name(symbol),
            market_metrics.get('Market Capitalization', 'N/A'),
            market_metrics.get('Beta', 'N/A'),
            market_metrics.get('Correlation', 'N/A'),
            market_metrics.get('Liquidity (Avg. Daily Volume)', 'N/A')
        ])

print("Market Metrics:")
print(market_table)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Market Metrics:
+------------------------------+------------------------+-------+-------------+-------------------------------+
|            Index             | Market Capitalization  |  Beta | Correlation | Liquidity (Avg. Daily Volume) |
+------------------------------+------------------------+-------+-------------+-------------------------------+
|           S&P 500            | 20,221,520,709,799.80  |  1.25 |     0.69    |        3,935,725,219.12       |
| Dow Jones Industrial Average | 15,970,078,414,687.50  |  1.08 |     0.50    |         325,373,984.06        |
|       NASDAQ Composite       | 199,254,376,960,937.50 |  0.98 |     0.74    |        5,049,552,908.37       |
|     NYSE COMPOSITE (DJ)      | 69,926,051,909,238.28  |  0.74 |     0.38    |        3,935,725,219.12       |
|  NYSE AMEX COMPOSITE INDEX   |          0.00          |  0.06 |     0.04    |              0.00             |
|         Russell 2000         |  8,002,371,487,500.00  |  0.26 |     0.24    |        3

In [110]:
def calculate_sma(stock_ticker, window):
    # Calculate start date as two years ago from today
    start_date = (datetime.datetime.now() - datetime.timedelta(days=365 * 2)).strftime('%Y-%m-%d')
    
    # Get today's date
    end_date = datetime.datetime.now().strftime('%Y-%m-%d')
    
    # Download historical price data for the stock
    stock_data = yf.download(stock_ticker, start=start_date, end=end_date)
    
    # Check if data is available
    if stock_data.empty:
        print(f"No price data available for {stock_ticker} within the specified time frame.")
        return None
    
    # Calculate Simple Moving Average (SMA)
    sma = stock_data['Adj Close'].rolling(window=window).mean()
    
    return sma

def get_fifty_day_average(stock_ticker):
    stock_info = yf.Ticker(stock_ticker)
    fifty_day_average = stock_info.info.get('fiftyDayAverage', None)
    return format_large_number("{:.2f}".format(fifty_day_average)) if fifty_day_average is not None else 'N/A'

def get_two_hundred_day_average(stock_ticker):
    stock_info = yf.Ticker(stock_ticker)
    two_hundred_day_average = stock_info.info.get('twoHundredDayAverage', None)
    return format_large_number("{:.2f}".format(two_hundred_day_average)) if two_hundred_day_average is not None else 'N/A'

sma_table = PrettyTable()
sma_table.field_names = ['Name', 'SMA 50', 'SMA 200', '50-Day Average', '200-Day Average']

choice_sma_50 = calculate_sma(choice, window=50)
choice_sma_200 = calculate_sma(choice, window=200)

if choice_sma_50 is not None and choice_sma_200 is not None:
    sma_table.add_row([get_ticker_name(choice), format_large_number("{:.2f}".format(choice_sma_50.iloc[-1])), format_large_number("{:.2f}".format(choice_sma_200.iloc[-1])), get_fifty_day_average(choice), get_two_hundred_day_average(choice)])

for symbol in stock_country_symbols:
    sma_50 = calculate_sma(symbol, window=50)
    sma_200 = calculate_sma(symbol, window=200)

    if sma_50 is not None and sma_200 is not None:
        sma_table.add_row([get_ticker_name(symbol), format_large_number("{:.2f}".format(sma_50.iloc[-1])), format_large_number("{:.2f}".format(sma_200.iloc[-1])), get_fifty_day_average(symbol), get_two_hundred_day_average(symbol)])

print("Simple Moving Averages:")
print(sma_table)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Simple Moving Averages:
+------------------------------+--------------+------------+----------------+-----------------+
|             Name             |    SMA 50    |  SMA 200   | 50-Day Average | 200-Day Average |
+------------------------------+--------------+------------+----------------+-----------------+
|    Microsoft Corporation     |    413.82    |   372.57   |     414.53     |      373.96     |
|           S&P 500            |   5,154.23   |  4,733.55  |    5,154.23    |     4,733.55    |
| Dow Jones Industrial Average |  38,817.58   | 36,587.20  |   38,817.58    |    36,587.20    |
|       NASDAQ Composite       |  16,128.12   | 14,768.24  |   16,128.12    |    14,768.24    |
|     NYSE COMPOSITE (DJ)      |  17,928.99   | 16,631.82  |   17,928.99    |    16,631.82    |
|  NYSE AMEX COMPOSITE INDEX   |   4,852.98   |  4,600.04  |    4,852.98    |     4,600.04    |
|         Russell 2000         |   2,042.80   |  1,919.94  |    2,042.80    |     1,919.94    |
|    CBOE Volati

In [111]:
def get_top_n_indices(stock_symbols, n, selected_period_value):
    stocks_info = []
    for symbol in stock_symbols:
        try:
            stock_data = yf.Ticker(symbol)
            if 'marketCap' in stock_data.info:
                market_cap = stock_data.info['marketCap']
                stocks_info.append({'Symbol': symbol, 'MarketCap': market_cap})
            elif stock_data.info['quoteType'] == 'INDEX':
                index_data = yf.download(symbol, period = selected_period_value)
                if not index_data.empty:
                    index_market_cap = index_data['Adj Close'].iloc[-1] * index_data['Volume'].iloc[-1]
                    stocks_info.append({'Symbol': symbol, 'MarketCap': index_market_cap})
        except Exception as e:
            print(f"Error processing symbol {symbol}: {e}")

    stocks_df = pd.DataFrame(stocks_info)

    if not stocks_df.empty:
        top_n_indices = stocks_df.nlargest(n, 'MarketCap')['Symbol'].tolist()
        return top_n_indices
    else:
        return []
    
def plot_stock_price_change_percentage(stock_tickers, period, label):
    fig = go.Figure()
    
    for stock_ticker in stock_tickers:
        if period == '1d':
            data = yf.download(tickers=stock_ticker, period=period, interval='60m')
        else:
            data = yf.download(tickers=stock_ticker, period=period)
        
        # Calculate percentage change in closing price
        data['Close_Percentage_Change'] = data['Close'].pct_change() * 100
        
        # Add trace for closing price change percentage
        fig.add_trace(go.Scatter(x=data.index, y=data['Close_Percentage_Change'], mode='lines', name=get_ticker_name(stock_ticker)))
    
    # Update layout
    fig.update_layout(
        title=f"Percentage Change in Closing Price for {label}",
        xaxis_title="Time",
        yaxis_title="Percentage Change",
        height=400,
        width=800,
    )

    # Show plot
    fig.show()

# Period labels and corresponding values
labels = ["1 day", "5 days", "1 month", "6 months", "year to date", "1 year", "5 years", "max"]
period_values = ["1d", "5d", "1mo", "6mo", "ytd", "1y", "5y", "max"]

# Randomly select one period
selected_period_label = random.choice(labels)
selected_period_value = period_values[labels.index(selected_period_label)]

# Take only top 5 stocks with the largest market capitalization
top_5_stocks = get_top_n_indices(stock_country_symbols, 5, selected_period_value)

# Combine choice and top 5 stocks
stock_tickers = [choice] + top_5_stocks

# Plot the stock price change percentage chart
plot_stock_price_change_percentage(stock_tickers, selected_period_value, selected_period_label)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [112]:
def fetch_news(choice, api_key):
    url = f"https://api.polygon.io/v2/reference/news?ticker={choice}&apiKey={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch news:", response.text)
        return None
        
def extract_tickers(news_data):
    tickers_list = []

    if news_data:
        for article in news_data['results']:
            tickers_list.extend(article['tickers'])
        
    return tickers_list

def extract_related_tickers(news_data):
    related_tickers = []

    if news_data:
        for article in news_data:
            related_tickers.extend(article.get('relatedTickers', []))

    return related_tickers

In [113]:
def get_top_n_peers(symbol, n):
    peers_info = []
    stock = yf.Ticker(symbol)
    api_key = "V2jiIqWR6ZKdcV0BC_sh84w6ZuaxArdE" 
    
    news_data1 = fetch_news(symbol, api_key)
    tickers_list = extract_tickers(news_data1)
    
    news_data2 = stock.news
    related_tickers = extract_related_tickers(news_data2)
    
    combined_tickers = list(set(related_tickers + tickers_list))
    if symbol in combined_tickers:
        combined_tickers.remove(symbol)
    
    for peer_symbol in combined_tickers:
        try:
            peer_data = yf.Ticker(peer_symbol)
            if 'marketCap' in peer_data.info and peer_data.info['quoteType'] == 'EQUITY' and (stock.info['industry'] == peer_data.info['industry'] or stock.info['sector'] == peer_data.info['sector']):
                market_cap = peer_data.info['marketCap']
                peers_info.append({'Symbol': peer_symbol, 'MarketCap': market_cap})
        except Exception as e:
            print(f"Error processing peer symbol {peer_symbol}: {e}")

    peers_df = pd.DataFrame(peers_info)

    if not peers_df.empty:
        top_n_peers = peers_df.nlargest(n, 'MarketCap')['Symbol'].tolist()
        return top_n_peers
    else:
        return []

def plot_stock_peer_price_change_percentage(stock_tickers, selected_time_period, selected_slider_value):
    fig = go.Figure()
    
    for stock_ticker in stock_tickers:
        if selected_time_period == '1d':
            stock_data = yf.download(tickers=stock_ticker, period=selected_time_period, interval='60m')
        else:
            stock_data = yf.download(tickers=stock_ticker, period=selected_time_period)
        
        stock_data['Close_Percentage_Change'] = stock_data['Close'].pct_change() * 100
        
        fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close_Percentage_Change'], mode='lines', name=get_ticker_name(stock_ticker)))
    
    fig.update_layout(
        title=f"Percentage Change in Closing Price for {selected_slider_value}",
        xaxis_title="Time",
        yaxis_title="Percentage Change",
        height=400,
        width=800,
    )
    
    fig.show()
    plot_html = pio.to_html(fig, full_html=False)
    return plot_html

top_5_peers = get_top_n_peers(choice, 5)
stock_tickers_1 = [choice] + top_5_peers
plot2_html = plot_stock_peer_price_change_percentage(stock_tickers_1, '1d', '1 day')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [114]:
balance_sheet = stock.balance_sheet
period = 'Annual'

date_strings = list(balance_sheet.keys())
financials_components = {extract_date(date): {} for date in date_strings}
components = ['Total Liabilities Net Minority Interest', 'Total Assets']

for date in date_strings:
    for component in components:
        try:
            financials_components[extract_date(date)][component] = balance_sheet[pd.Timestamp(date)][component]
        except KeyError:
            financials_components[extract_date(date)][component] = None

dates = [extract_date(date) for date in date_strings]
years = [date.strftime('%Y') for date in date_strings]
liabilities_data = [financials_components[date]['Total Liabilities Net Minority Interest'] for date in dates]
assets_data = [financials_components[date]['Total Assets'] for date in dates]

trace1 = go.Bar(
    x=years,
    y=liabilities_data,
    name='Total Liabilities'
)

trace2 = go.Bar(
    x=years,
    y=assets_data,
    name='Total Assets'
)

data = [trace1, trace2]

layout = go.Layout(
    barmode='group',
    title='Total Liabilities and Total Assets Over Time (Annual)',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Amount'),
    height=400,
    width=800
)

fig = go.Figure(data=data, layout=layout)
fig.show()

In [115]:
balance_sheet = stock.quarterly_balance_sheet
period = 'Quarterly'

date_strings = list(balance_sheet.keys())
financials_components = {extract_date(date): {} for date in date_strings}
components = ['Total Liabilities Net Minority Interest', 'Total Assets']

for date in date_strings:
    for component in components:
        try:
            financials_components[extract_date(date)][component] = balance_sheet[pd.Timestamp(date)][component]
        except KeyError:
            financials_components[extract_date(date)][component] = None

dates = [extract_date(date) for date in date_strings]
quarters = [f"{(date.month - 1) // 3 + 1}Q{date.strftime('%Y')}" for date in date_strings]
liabilities_data = [financials_components[date]['Total Liabilities Net Minority Interest'] for date in dates]
assets_data = [financials_components[date]['Total Assets'] for date in dates]

trace1 = go.Bar(
    x=quarters,
    y=liabilities_data,
    name='Total Liabilities'
)

trace2 = go.Bar(
    x=quarters,
    y=assets_data,
    name='Total Assets'
)

data = [trace1, trace2]

layout = go.Layout(
    barmode='group',
    title='Total Liabilities and Total Assets Over Time (Quarterly)',
    xaxis=dict(title='Quarter'),
    yaxis=dict(title='Amount'),
    height=400,
    width=800
)

fig = go.Figure(data=data, layout=layout)
fig.show()

In [116]:
income_statement = stock.income_stmt
period = 'Annual'

date_strings = list(income_statement.keys())
financials_components = {extract_date(date): {} for date in date_strings}
components = ['Net Income', 'Total Revenue']

for date in date_strings:
    for component in components:
        try:
            financials_components[extract_date(date)][component] = income_statement[pd.Timestamp(date)][component]
        except KeyError:
            financials_components[extract_date(date)][component] = None

dates = [extract_date(date) for date in date_strings]
years = [date.strftime('%Y') for date in date_strings]
net_income_data = [financials_components[date]['Net Income'] for date in dates]
revenue_data = [financials_components[date]['Total Revenue'] for date in dates]

trace1 = go.Bar(
    x=years,
    y=net_income_data,
    name='Net Income'
)

trace2 = go.Bar(
    x=years,
    y=revenue_data,
    name='Total Revenue'
)

data = [trace1, trace2]

layout = go.Layout(
    barmode='group',
    title='Net Income and Total Revenue Over Time (Annual)',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Amount'),
    height=400,
    width=800
)

fig = go.Figure(data=data, layout=layout)
fig.show()

In [117]:
income_statement = stock.quarterly_income_stmt
period = 'Quarterly'

date_strings = list(income_statement.keys())
financials_components = {extract_date(date): {} for date in date_strings}
components = ['Net Income', 'Total Revenue']

for date in date_strings:
    for component in components:
        try:
            financials_components[extract_date(date)][component] = income_statement[pd.Timestamp(date)][component]
        except KeyError:
            financials_components[extract_date(date)][component] = None

dates = [extract_date(date) for date in date_strings]
quarters = [f"{(date.month - 1) // 3 + 1}Q{date.strftime('%Y')}" for date in date_strings]
net_income_data = [financials_components[date]['Net Income'] for date in dates]
revenue_data = [financials_components[date]['Total Revenue'] for date in dates]

trace1 = go.Bar(
    x=quarters,
    y=net_income_data,
    name='Net Income'
)

trace2 = go.Bar(
    x=quarters,
    y=revenue_data,
    name='Total Revenue'
)

data = [trace1, trace2]

layout = go.Layout(
    barmode='group',
    title='Net Income and Total Revenue Over Time (Quarterly)',
    xaxis=dict(title='Quarter'),
    yaxis=dict(title='Amount'),
    height=400,
    width=800
)

fig = go.Figure(data=data, layout=layout)
fig.show()

In [118]:
cashflow = stock.cashflow
period = 'Annual'

date_strings = list(cashflow.keys())
financials_components = {extract_date(date): {} for date in date_strings}
components = ['Changes In Cash']

for date in date_strings:
    for component in components:
        try:
            financials_components[extract_date(date)][component] = cashflow[pd.Timestamp(date)][component]
        except KeyError:
            financials_components[extract_date(date)][component] = None

dates = [extract_date(date) for date in date_strings]
years = [date.strftime('%Y') for date in date_strings]
net_changes_in_cash_data = [financials_components[date]['Changes In Cash'] for date in dates]

trace = go.Bar(
    x=years,
    y=net_changes_in_cash_data,
    name='Net Changes In Cash'
)

layout = go.Layout(
    title='Net Changes In Cash Over Time (Annual)',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Amount'),
    height=400,
    width=800
)

fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [119]:
cashflow = stock.quarterly_cashflow
period = 'Quarterly'

date_strings = list(cashflow.keys())
financials_components = {extract_date(date): {} for date in date_strings}
components = ['Changes In Cash']

for date in date_strings:
    for component in components:
        try:
            financials_components[extract_date(date)][component] = cashflow[pd.Timestamp(date)][component]
        except KeyError:
            financials_components[extract_date(date)][component] = None

dates = [extract_date(date) for date in date_strings]
quarters = [f"{(date.month - 1) // 3 + 1}Q{date.strftime('%Y')}" for date in date_strings]
net_changes_in_cash_data = [financials_components[date]['Changes In Cash'] for date in dates]

trace = go.Bar(
    x=quarters,
    y=net_changes_in_cash_data,
    name='Net Changes In Cash'
)

layout = go.Layout(
    title='Net Changes In Cash Over Time (Quarterly)',
    xaxis=dict(title='Quarter'),
    yaxis=dict(title='Amount'),
    height=400,
    width=800
)

fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [120]:
holders = get_holders(choice)

try:
    major_holders = holders['Major Holders']

    major_holders_df = pd.DataFrame(major_holders)
    major_holders_df.columns = ['Breakdown', 'Description']

    major_holders_table = PrettyTable()
    major_holders_table.field_names = ['Breakdown', 'Description']

    for row in major_holders_df.itertuples(index=False):
        major_holders_table.add_row(row)

    print(major_holders_table)
except KeyError:
    pass

+-----------+---------------------------------------+
| Breakdown |              Description              |
+-----------+---------------------------------------+
|   0.05%   |    % of Shares Held by All Insider    |
|   73.65%  |    % of Shares Held by Institutions   |
|   73.69%  |    % of Float Held by Institutions    |
|    6833   | Number of Institutions Holding Shares |
+-----------+---------------------------------------+


In [121]:
try:
    direct_holders = holders['Direct Holders (Forms 3 and 4)']

    direct_holders_table = PrettyTable()
    direct_holders_table.field_names = ["Holder", "Shares Held", "Percentage Held", "Value"]

    for index, row in direct_holders.iterrows():
        holder = row['Holder']
        shares_held = row['Shares']
        percentage = row['% Out']
        value = format_large_number(row['Value'])
        direct_holders_table.add_row([holder, shares_held, percentage, value]) 
        
    print(direct_holders_table)
except KeyError:
    pass

+-----------------------------------+-------------+-----------------+-----------------+
|               Holder              | Shares Held | Percentage Held |      Value      |
+-----------------------------------+-------------+-----------------+-----------------+
|         Vanguard Group Inc        |   667.99M   |      8.99%      | 280,193,837,251 |
|           Blackrock Inc.          |   546.02M   |      7.35%      | 229,033,485,390 |
|      State Street Corporation     |   300.35M   |      4.04%      | 125,986,583,168 |
|              FMR, LLC             |   220.86M   |      2.97%      |  92,641,806,196 |
|   Geode Capital Management, LLC   |   157.59M   |      2.12%      |  66,102,145,527 |
|   Price (T.Rowe) Associates Inc   |    142.2M   |      1.91%      |  59,647,497,276 |
|     JP Morgan Chase & Company     |   127.74M   |      1.72%      |  53,582,302,526 |
|           Morgan Stanley          |   122.19M   |      1.64%      |  51,252,607,472 |
| Norges Bank Investment Managem

In [122]:
try:
    institutional_holders = holders['Top Institutional Holders']

    institutional_holders_table = PrettyTable()
    institutional_holders_table.field_names = ["Holder", "Shares Held", "Percentage Held", "Value"]

    for index, row in institutional_holders.iterrows():
        holder = row['Holder']
        shares_held = row['Shares']
        percentage = row['% Out']
        value = format_large_number(row['Value'])
        institutional_holders_table.add_row([holder, shares_held, percentage, value])

    print(institutional_holders_table)
except KeyError:
    pass

+------------------------------------------------------------+-------------+-----------------+----------------+
|                           Holder                           | Shares Held | Percentage Held |     Value      |
+------------------------------------------------------------+-------------+-----------------+----------------+
|           Vanguard Total Stock Market Index Fund           |   233.27M   |      3.14%      | 97,846,042,955 |
|                  Vanguard 500 Index Fund                   |   181.56M   |      2.44%      | 76,155,186,264 |
|                   SPDR S&P 500 ETF Trust                   |    89.66M   |      1.21%      | 37,610,437,603 |
|                  Fidelity 500 Index Fund                   |    89.8M    |      1.21%      | 37,668,284,072 |
|                  iShares Core S&P 500 ETF                  |    77.03M   |      1.04%      | 32,309,158,356 |
|                 Vanguard Growth Index Fund                 |    68.13M   |      0.92%      | 28,578,94

In [124]:
def fetch_news(choice, api_key='V2jiIqWR6ZKdcV0BC_sh84w6ZuaxArdE'):
    url = f"https://api.polygon.io/v2/reference/news?ticker={choice}&apiKey={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch news:", response.text)
        return None
    
def read_article(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        print("Error occurred:", str(e))
        return None

def preprocess_text(text):
    text = text.lower()

    text = ''.join([c for c in text if c not in punctuation])

    tokens = word_tokenize(text)

    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text
    
def get_sentiment(text):
    blob = TextBlob(text)
    sentiment_score = "{:.3f}".format(blob.sentiment.polarity)
    sentiment_score = float(sentiment_score)
    if sentiment_score > 0:
        sentiment = 'positive'
    elif sentiment_score < 0:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'
    return sentiment_score, sentiment

news_data = []
news_items = fetch_news(choice, api_key)

if news_items and 'results' in news_items:
    for item in news_items['results']:
        if 'article_url' in item:
            article_url = item['article_url']
            article_text = read_article(article_url)
            if article_text:
                preprocessed_text = preprocess_text(article_text)
                sentiment_score, sentiment = get_sentiment(preprocessed_text)
                news_data.append({'Article_URL': article_url, 'Article_Title': item['title'], 'Sentiment_Score': sentiment_score, 'Sentiment': sentiment})
            else:
                print("Failed to read article from URL:", article_url)
else:
    print("Failed to fetch news items or results not found.")

news_df = pd.DataFrame(news_data)
n_samples = min(5, len(news_df))
news_df = news_df.sample(n=n_samples, replace=False, random_state=None)
news_df.reset_index(drop=True, inplace=True)
print(news_df)

                                         Article_URL  \
0  https://www.fool.com/investing/2024/05/16/alph...   
1  https://www.zacks.com/stock/news/2275166/is-fl...   
2  https://www.fool.com/investing/2024/05/17/why-...   
3  https://www.zacks.com/stock/news/2274895/alpha...   
4  https://www.fool.com/investing/2024/05/17/can-...   

                                       Article_Title  Sentiment_Score  \
0  Alphabet Just Announced Spectacular News for N...           -0.015   
1  Is FlexShares STOXX US ESG Select Index Fund (...           -0.219   
2                      Why Reddit Stock Soared Today            0.050   
3  Alphabet (GOOGL) Boosts Google TV With Generat...           -0.219   
4                         Can Shiba Inu Reach $0.10?            0.029   

  Sentiment  
0  negative  
1  negative  
2  positive  
3  negative  
4  positive  
